# Route Optimization with Chicago Dataset

### Import libraries

In [ ]:
!pip install dask

In [ ]:
!pip install scikit-plot

In [ ]:
!pip install statsmodels

In [ ]:
!pip install prophet

In [ ]:
import pandas as pd  
import numpy as np  

import matplotlib.pyplot as plt 
import seaborn as sns  
import folium  
from folium import plugins
from folium import Choropleth, Circle, Marker
#from folium.plugins import TimeSliderChoropleth
import plotly.express as px
import json
import osmnx as ox  
import networkx as nx 
import geopandas as gpd

import re
import random

from sklearn.cluster import KMeans 
from scikitplot.cluster import plot_elbow_curve

import dask.dataframe as dd
from statsmodels.tsa.seasonal import seasonal_decompose
import gc

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, median_absolute_error, r2_score, make_scorer
from skopt import BayesSearchCV
from prophet import Prophet
from sklearn.preprocessing import StandardScaler
from prophet.diagnostics import cross_validation, performance_metrics
from sklearn.model_selection import TimeSeriesSplit
import joblib

# Table of Contents
- [Chapter 1 Match Real Trip Routes with Calculated Shortest Routes from OSMnx](#Chapter-1-Match-Real-Trip-Routes-with-Calculated-Shortest-Routes-from-OSMnx)
  - [1.1 Data preparation and OSMnx graph creation](#1.1-Data-preparation-and-OSMnx-graph-creation)
  - [1.2 Match the end points of each trip with the nodes from the OSMnx graph and calculate the shortest route with the obtained nodes](#1.2-Match-the-end-points-of-each-trip-with-the-nodes-from-the-OSMnx-graph-and-calculate-the-shortest-route-with-the-obtained-nodes)
  - [1.3 Filter trips based on the match result](#1.3-Filter-trips-based-on-the-match-result)
    - [1.3.1 Filter trips by the point-to-nearest-node distance](#1.3.1-Filter-trips-by-the-point-to-nearest-node-distance)
    - [1.3.2 Filter trips by the absolute difference between the real trip miles and the calculated shortest route length](#1.3.2-Filter-trips-by-the-absolute-difference-between-the-real-trip-miles-and-the-calculated-shortest-route-length)
      - [1.3.2.1 compare the trip features before and after filter with different thresholds](#1.3.2.1-compare-the-trip-features-before-and-after-filter-with-different-thresholds)
      - [1.3.2.2 compare the temporal traffic pattern before and after filter with the 500m threshold](#1.3.2.2-compare-the-temporal-traffic-pattern-before-and-after-filter-with-the-500m-threshold)    
- [Chapter 2 Create Segment Level Dataset](#Chapter-2-Create-Segment-Level-Dataset)
  - [2.1 Cluster all road segments of Chicago into regions](#2.1-Cluster-all-road-segments-of-Chicago-into-regions)
  - [2.2 Extract the road segments details from G](#2.2-Extract-the-road-segments-details-from-G)
  - [2.3 Display the road segments in our dataset on OSMnx graph and check their coverage ratio in each region](#2.3-Display-the-road-segments-in-our-dataset-on-OSMnx-graph-and-check-their-coverage-ratio-in-each-region)
  - [2.4 Complete the segment level traffic information](#2.4-Complete-the-segment-level-traffic-information)
  
- [Chapter 3 Check the Temporal Traffic Pattern Completeness for Each Region](#Chapter-3-Check-the-Temporal-Traffic-Pattern-Completeness-for-Each-Region)
- [Chapter 4 Road Segments Clustering within Each Region](#Chapter-4-Road-Segments-Clustering-within-Each-Region)
  - [4.1 Data preparation for KMeans clustering](#4.1-Data-preparation-for-KMeans-clustering)
  - [4.2 Use elbow method to determine K for each region](#4.2-Use-elbow-method-to-determine-K-for-each-region)
  - [4.3 Save the obtained cluster information for each region to the segment level dataframe](#4.3-Save-the-obtained-cluster-information-for-each-region-to-the-segment-level-dataframe)
  - [4.4 Display the above plotted results onto map](#4.4-Display-the-above-plotted-results-onto-map)
    - [4.4.1 Region data preparation](#4.4.1-Region-data-preparation)
    - [4.4.2 Plot road segments on map, colored by congestion level](#4.4.2-Plot-road-segments-on-map,-colored-by-congestion-level)
- [Chapter 5  Build ML models for predicting the  travel time for each road segment cluster](#Chapter-5--Build-ML-models-for-predicting-the--travel-time-for-each-road-segment-cluster)
  - [5.1 Check the clustering accuracy by plotting the inferred travel time histogram for each cluster](#5.1-Check-the-clustering-accuracy-by-plotting-the-inferred-travel-time-histogram-for-each-cluster)
  - [5.2 EDA for feature engineering](#5.2-EDA-for-feature-engineering)
    - [5.2.1 fill up missing hours](#5.2.1-fill-up-missing-hours)
    - [5.2.2 Generate lag and rolling features](#5.2.2-Generate-lag-and-rolling-features)
    - [5.2.3 EDA on individual features and correlation between features](#5.2.3-EDA-on-individual-features-and-correlation-between-features)
      - [5.2.3.1 Exploring single numerical features](#5.2.3.1-Exploring-single-numerical-features)
      - [5.2.3.2 Exploring the correlation between features](#5.2.3.2-Exploring-the-correlation-between-features)
      - [5.2.3.3 Use Xgboost as the base model to do feature engneering](#5.2.3.3-Use-Xgboost-as-the-base-model-to-do-feature-engneering)
   - [5.3 Machine Learning Modeling](#5.3-Machine-Learning-Modeling)
     - [5.3.1 Xgboost model with Bayesian optimization (BayesSearchCV)](#5.3.1-Xgboost-model-with-Bayesian-optimization-(BayesSearchCV))
     - [5.3.2 Prophet model](#5.3.2-Prophet-model)
   - [5.4 Use the prediction results to do route optimization and compare the results with that based on the inferred travel time](#5.4-Use-the-prediction-results-to-do-route-optimization-and-compare-the-results-with-that-based-on-the-inferred-travel-time)
     - [5.4.1 Merge the prediction results for each cluster to the original regional dataset](#5.4.1-Merge-the-prediction-results-for-each-cluster-to-the-original-regional-dataset)
     - [5.4.2 Compare the route optimization results by using the predition results on cluster level and the inferred travel time on segment level](#5.4.2-Compare-the-route-optimization-results-by-using-the-predition-results-on-cluster-level-and-the-inferred-travel-time-on-segment-level)
       - [5.4.2.1 Analyze the difference between the inferred_travel_time_sec and the predicted travel time for each segment](#5.4.2.1-Analyze-the-difference-between-the-inferred_travel_time_sec-and-the-predicted-travel-time-for-each-segment)
       - [5.4.2.2 Compare the route optimization results by the prediction and the inferred travel time](#5.4.2.2-Compare-the-route-optimization-results-by-the-prediction-and-the-inferred-travel-time)
      

## Chapter 1 Match Real Trip Routes with Calculated Shortest Routes from OSMnx

### 1.1 Data preparation and OSMnx graph creation

In [ ]:
# import chicago data
df = pd.read_csv('chicago_data_5.2_perc_sample.csv')
# drop na values in pickup_centroid_location or dropoff_centroid_location
data=df.dropna(subset=['pickup_centroid_location', 'dropoff_centroid_location'])

In [ ]:
# create an OSMnx graph for Chicago city
place = 'Chicago, Illinois, USA'
G = ox.graph_from_place(place, network_type='drive')

#add speed limit and free flow travel_time attributes to the edges.
G = ox.add_edge_speeds(G)
G = ox.add_edge_travel_times(G)
# create geo-dataframe for both nodes and edges 
nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)

### 1.2 Match the end points of each trip with the nodes from the OSMnx graph and calculate the shortest route with the obtained nodes

In [ ]:
# create a dataframe data_group to only include the unique ('pickup_centroid_location', 'dropoff_centroid_location') 
data_group=data.drop_duplicates(['pickup_centroid_location', 'dropoff_centroid_location'])

In [ ]:
#convert the end points of each pair into nodes of the graph:
data_group['start_node']=data_group.apply(lambda row: 
                                                        ox.distance.nearest_nodes(
                                                            G, 
                                                            float(re.findall(pattern, row['pickup_centroid_location'])[0]),
                                                            float(re.findall(pattern, row['pickup_centroid_location'])[1])),
                                                            axis=1)
data_group['end_node']=data_group.apply(lambda row: 
                                                        ox.distance.nearest_nodes(
                                                            G, 
                                                            float(re.findall(pattern, row['dropoff_centroid_location'])[0]),
                                                            float(re.findall(pattern, row['dropoff_centroid_location'])[1])),
                                                            axis=1)

In [ ]:
# Calculate the distance between the point and its nearest node
data_group['start_node_distance'] = data_group.apply(lambda row: 
                                                                       ox.distance.great_circle_vec(
                                                                       float(re.findall(pattern, row['pickup_centroid_location'])[1]),
                                                                       float(re.findall(pattern, row['pickup_centroid_location'])[0]),
                                                                       G.nodes[row['start_node']]['y'], G.nodes[row['start_node']]['x']),
                                                                   axis=1)
data_group['end_node_distance'] = data_group.apply(lambda row: 
                                                                       ox.distance.great_circle_vec(
                                                                       float(re.findall(pattern, row['dropoff_centroid_location'])[1]),
                                                                       float(re.findall(pattern, row['dropoff_centroid_location'])[0]),
                                                                       G.nodes[row['end_node']]['y'], G.nodes[row['end_node']]['x']),
                                                                   axis=1)

In [ ]:
# calculate the shortest route by length for each pair of the obtained nodes 
def get_shortest_route(row):
    try:
        shortest_route=nx.shortest_path(G, row['start_node'], row['end_node'], weight='length')
        shortest_route_length=nx.shortest_path_length(G, row['start_node'], row['end_node'], weight='length')   
    except nx.NetworkXNoPath:
        shortest_route=np.nan
        shortest_route_length=np.nan
    return shortest_route, shortest_route_length
data_group[['shortest_route','shortest_route_length']]= data_group.apply(
                                                                                                                                 lambda row:
                                                                                                                                pd.Series(get_shortest_route(row)), axis=1)

In [ ]:
# merge the new generated columns in above with the original data dataframe and rename it as data_merged
data_group_col=data_group[['pickup_centroid_location', 'dropoff_centroid_location', 'start_node','end_node',
                           'start_node_distance', 'end_node_distance','shortest_route', 'shortest_route_length']]
data_merged=pd.merge(data, data_group_col, on=['pickup_centroid_location', 'dropoff_centroid_location'])

In [ ]:
# save this dataframe for an easier access in the future
data_merged.to_csv('Chicago_orig.csv')

In [ ]:
data_merged=pd.read_csv('Chicago_orig.csv')

### 1.3 Filter trips based on the match result 

#### 1.3.1 Filter trips by the point-to-nearest-node distance

In [ ]:
# obtain the statistics info of the distance between the pickup point and the found nearest node
data_merged['start_node_distance'].describe().drop('count')

In [ ]:
# obtain the statistics info of the distance between the dropoff point and the found nearest node
data_merged['end_node_distance'].describe().drop('count')

Analysis: 
- the distance of ~82m between the closest node and the pickup/dropoff location corresponds to 75 percentile, we can start with  82m as the threshold to filter trips, and then try other thresholds based on the result.

In [ ]:
# calculate the retained data ratio by trying different thresholds
data_ratio_thre_82m=len(data_merged[(data_merged['start_node_distance']<82) 
                         & (data_merged['end_node_distance']<82)])/len(data_merged)
data_ratio_thre_100m=len(data_merged[(data_merged['start_node_distance']<100) & 
                (data_merged['end_node_distance']<100)])/len(data_merged)
data_ratio_thre_150m=len(data_merged[(data_merged['start_node_distance']<150) 
                & (data_merged['end_node_distance']<150)])/len(data_merged)
data_ratio_thre_200m=len(data_merged[(data_merged['start_node_distance']<200) 
                & (data_merged['end_node_distance']<200)])/len(data_merged)
data_ratio_thre_500m=len(data_merged[(data_merged['start_node_distance']<500) 
                & (data_merged['end_node_distance']<500)])/len(data_merged)
data_ratio_thre_1000m=len(data_merged[(data_merged['start_node_distance']<1000) 
                                     & (data_merged['end_node_distance']<1000)])/len(data_merged)
print(f'data_ratio_thre_82m: {data_ratio_thre_82m},\
    data_ratio_thre_100m: {data_ratio_thre_100m},\
     data_ratio_thre_150m: {data_ratio_thre_150m},\
     data_ratio_thre_200m: {data_ratio_thre_200m},\
     data_ratio_thre_500m: {data_ratio_thre_500m},\
     data_ratio_thre_1000m: {data_ratio_thre_1000m}')

Analysis:
- from the above result, we can see that since the threshold of 150m, the increase of the retained data ratio becomes insignificant, and by considering that 150m distance is still reasonable, we can conclude that 150m should be the best threshod to filter trips so as to keep as many as possible trips reasonably. 

In [ ]:
 # create a new dataframe called data_merged_nodes reflecting the filter by nodes as mentioned above. 
data_merged_nodes=data_merged[(data_merged['start_node_distance']<150) &
                                                                     (data_merged['end_node_distance']<150)  &
                                                                     (data_merged['start_node'] != data_merged['end_node'])
                              ]
# drop the invalid routes 
data_merged_nodes.dropna(subset=['shortest_route'], inplace=True)
# add a new column about the absolute difference between the actual trip_miles and the calculated shortest route length
data_merged_nodes['absdiff_trip_shortest_meters']=abs(data_merged_nodes['trip_miles']*1609.344-data_merged_nodes['shortest_route_length'])

#### 1.3.2 Filter trips by the absolute difference between the real trip miles and the calculated shortest route length

In [ ]:
# obtain the statistics info of the absolute difference between the real trip miles and the calcuated shortest route length
data_merged_nodes['absdiff_trip_shortest_meters'].describe().drop('count')

In [ ]:
# plot the boxplot by focusing on the difference range within 2500 for a better visualization
Y=data_merged_nodes[data_merged_nodes['absdiff_trip_shortest_meters']<2500]
Y['absdiff_trip_shortest_meters'].plot(kind='box')

Analysis:
- it seems we can consider 1610 as the maximum threshold since it retains 75% data.
- we can also try 1000m and 500m for comparison since both thresholds retain at least ~50% data and are more reasonable than 1610m.

In [ ]:
# data_merged_routes_diffXXX was obtained after filtering trips with the threshold of XXXm 
data_merged_routes_diff1610=data_merged_nodes[data_merged_nodes['absdiff_trip_shortest_meters']<1610]
data_merged_routes_diff500=data_merged_nodes[data_merged_nodes['absdiff_trip_shortest_meters']<500]
data_merged_routes_diff1000=data_merged_nodes[data_merged_nodes['absdiff_trip_shortest_meters']<1000]

##### 1.3.2.1 compare the trip features before and after filter with different thresholds

In [ ]:
filter_nodes=data_merged_nodes[['trip_miles', 'trip_seconds', 'percent_time_chicago', 
                                'percent_distance_chicago']].describe().drop('count')
filter_routes_diff500=data_merged_routes_diff500[['trip_miles', 'trip_seconds', 'percent_time_chicago',
                            'percent_distance_chicago']].describe().drop('count')
filter_routes_diff1000=data_merged_routes_diff1000[['trip_miles', 'trip_seconds', 'percent_time_chicago',
                            'percent_distance_chicago']].describe().drop('count')
filter_routes_diff1610=data_merged_routes_diff1610[['trip_miles', 'trip_seconds', 'percent_time_chicago',
                            'percent_distance_chicago']].describe().drop('count')
m1=filter_nodes.merge(filter_routes_diff500, left_index=True, right_index=True, suffixes=['_filter_nodes', '_filter_routes_diff500'])
m2=filter_routes_diff1000.merge(filter_routes_diff1610, left_index=True, right_index=True, suffixes=['_filter_routes_diff1000', '_filter_routes_diff1610'])
compare=m1.merge(m2, left_index=True, right_index=True)

In [ ]:
#trip_miles comparison
com_trip_miles=compare[['trip_miles_filter_nodes',
                        'trip_miles_filter_routes_diff500',
                        'trip_miles_filter_routes_diff1000',
                        'trip_miles_filter_routes_diff1610'
        ]]
com_trip_miles

In [ ]:
com_trip_seconds=compare[['trip_seconds_filter_nodes', 
                          'trip_seconds_filter_routes_diff500',
                          'trip_seconds_filter_routes_diff1000',
                          'trip_seconds_filter_routes_diff1610'
 ]]
com_trip_seconds

In [ ]:
com_percent_time_chicago=compare[['percent_time_chicago_filter_nodes',
                                  'percent_time_chicago_filter_routes_diff500',
                                  'percent_time_chicago_filter_routes_diff1000',
                                  'percent_time_chicago_filter_routes_diff1610'
     ]]
com_percent_time_chicago

In [ ]:
com_percent_distance_chicago=compare[['percent_distance_chicago_filter_nodes',
                                      'percent_distance_chicago_filter_routes_diff500',
                                      'percent_distance_chicago_filter_routes_diff1000',
                                      'percent_distance_chicago_filter_routes_diff1610'
   ]]
com_percent_distance_chicago

Analysis: 
- trip_miles statistical features have been changed obviously after the route filter with different thresholds. However, we can see that the 1610m threshod doesnt have a significant difference from the 500m threshod, which implies the retained trips by using the 500m threshold already kept most of the segment types. Thus, we can consider using the 500m threshod for the route filtering.
- trip_seconds statistical features dont have significant changes after the route filter with different thresholds, which makes sense because a lot of factors can contribute to the travel time, like the weather, the traffic, and some unexpected events, so even if we have lost some diversity of road types after the route filter, the trip_seconds might still remain similar.
- both percent_time_chicago and percent_distance_chicago are retained well after route filter with different thresholds, which makes sense if most of  the trips occurred within Chicago city limit. 

Conclusion:
- it seems 500m threshold works well by comparing the trip related features before and after filter. 

##### 1.3.2.2 compare the temporal traffic pattern before and after filter with the 500m threshold

In [ ]:
# convert the timestamp columns into datetime object
df['trip_start_timestamp']=pd.to_datetime(df['trip_start_timestamp'])
df['trip_end_timestamp']=pd.to_datetime(df['trip_end_timestamp'])
data_merged_routes_diff500['trip_start_timestamp']=pd.to_datetime(data_merged_routes_diff500['trip_start_timestamp'])
data_merged_routes_diff500['trip_end_timestamp']=pd.to_datetime(data_merged_routes_diff500['trip_end_timestamp'])

In [ ]:
# extract year, month, day of week, hour features.
df['year']=df['trip_start_timestamp'].dt.year
df['month']=df['trip_start_timestamp'].dt.month
df['dayname']=df['trip_start_timestamp'].dt.day_name()
df['hour_start']=df['trip_start_timestamp'].dt.hour
df['hour_end']=df['trip_end_timestamp'].dt.hour
data_merged_routes_diff500['year']=data_merged_routes_diff500['trip_start_timestamp'].dt.year
data_merged_routes_diff500['month']=data_merged_routes_diff500['trip_start_timestamp'].dt.month
data_merged_routes_diff500['dayname']=data_merged_routes_diff500['trip_start_timestamp'].dt.day_name()
data_merged_routes_diff500['hour_start']=data_merged_routes_diff500['trip_start_timestamp'].dt.hour
data_merged_routes_diff500['hour_end']=data_merged_routes_diff500['trip_end_timestamp'].dt.hour

In [ ]:
# compare the yearly temporal traffic pattern between the original dataset df without any filtering and the dataset after both nodes and routes filter
org_year=df['year'].value_counts(normalize=True)
routes500_year=data_merged_routes_diff500['year'].value_counts(normalize=True)
pd.DataFrame({'org_year': org_year, 'routes500_year': routes500_year})

In [ ]:
# compare the monthly temporal traffic pattern 
org_month=df['month'].value_counts(normalize=True).sort_index()
routes500_month=data_merged_routes_diff500['month'].value_counts(normalize=True)
pd.DataFrame({'org_month': org_month,  
              'routes500_month': routes500_month,
              })

In [ ]:
# compare the day of week temporal traffic pattern 
org_dayname=df['dayname'].value_counts(normalize=True).reindex(['Monday', 'Tuesday', 'Wednesday',
                                                    'Thursday', 'Friday','Saturday', 'Sunday'])
                                                                                               
routes500_dayname=data_merged_routes_diff500['dayname'].value_counts(normalize=True).reindex(['Monday',
                                                                                                'Tuesday', 'Wednesday','Thursday', 'Friday','Saturday', 'Sunday'])

pd.DataFrame({'org_dayname': org_dayname, 
              'routes500_dayname': routes500_dayname})

In [ ]:
# compare the hourly temporal traffic pattern 
org_hour_start=df['hour_start'].value_counts(normalize=True).sort_index()
routes500_hour_start=data_merged_routes_diff500['hour_start'].value_counts(normalize=True).sort_index()
pd.DataFrame({'org_hour_start': org_hour_start, 'routes500_hour_start': routes500_hour_start,
              })

Analysis:
- it seems the temporal pattern has been retained well after the filter.
- since the temporal features are well retained, the weather related features are also supposed to be well retained.
- obvious difference in trip related features but high similarity in temporal patterns implies the remained trips after filter are common trips with good representative. 

In [ ]:
# save the data filted by both nodes and routes to a csv file for an easier access in the future
data_merged_routes_diff500.to_csv('Chicago_routes_filter.csv')

In [ ]:
data_merged_routes_diff500=pd.read_csv('Chicago_routes_filter.csv')

In [ ]:
#averge trip duration in min 
trip_duration_min=data_merged_routes_diff500['trip_seconds'].mean()/60
trip_duration_min

## Chapter 2 Create Segment Level Dataset

### 2.1 Cluster all road segments of Chicago into regions

In [ ]:
# define a function to remove the list object 
def remove_list_obj (col):
    col=col.apply(lambda x: tuple(x) if isinstance(x, list) else x)
    return col
# apply this function to edges gdf for easier operation in the following
edges=edges.apply(remove_list_obj)

In [ ]:
# Extract centroids of each road segment in edges
edges['centroid'] = edges['geometry'].centroid
edges['x'] = edges['centroid'].x
edges['y'] = edges['centroid'].y

#Use the centroids' coordinates to cluster all road segments of Chicago into regions.
n_clusters = 29  
# Apply K-Means
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
edges['region'] = kmeans.fit_predict(edges[['x', 'y']])

**Note**: I used 29 because I referred to the region division information from website https://data.cityofchicago.org/api/assets/3F039704-BD76-4E6E-8E42-5F2BB01F0AF8

In [ ]:
# Iterate through the edges in edges gdf and update G with region cluster  so that we can extract this information from G in the following
edges=edges.reset_index()
for _, row in edges.iterrows():
    u, v, region = row['u'], row['v'], row['region']
    for key in G[u][v]:
        G[u][v][key]['region'] = region

In [ ]:
#add Geometry from edge GeoDataFrame to G for the plot in the following
for idx, row in edges.iterrows():
    u, v, key = row['u'], row['v'], row['key']
    G[u][v][key]['geometry'] = row['geometry']

In [ ]:
# Plot road segments on OSMnx, colored by their cluster (region)
fig, ax = plt.subplots(figsize=(12, 12))
# Calculate and plot region labels
region_centroids = edges.groupby('region')['geometry'].apply(lambda x: x.unary_union.centroid)

for region, centroid in region_centroids.items():
    ax.text(centroid.x, centroid.y, f'region{region}',  # Adjust position slightly
            fontsize=6, ha='center', va='center', color='black', 
            bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))

edges.plot(ax=ax, column='region', cmap='tab20', legend=True, linewidth=0.8, legend_kwds={'label': "Region Clusters"})

# Add title
plt.title('Road Segments Clustered into 29 Regions (K-Means)', fontsize=16)

### 2.2 Extract the road segments details from G 

In [ ]:
# create a new dataframe called data_merged_routes_diff500_chopped to only contain the unique routes
data_merged_routes_diff500_chopped=data_merged_routes_diff500.drop_duplicates(['shortest_route'])

In [ ]:
# define a function to extract the road segments details of each shortest route from G
def extract_road_segments_details(row):
   # if the data was obtained by reading a csv file, the list of nodes would be string. Otherwise, it is a list of integers and can be used directly.   
    if isinstance(row['shortest_route'], str):
        shortest_route_str=row['shortest_route']
        nodes_str=re.findall(r'\d+', shortest_route_str)
        nodes_int=[int(x) for x in nodes_str]
    else:
        nodes_int=row['shortest_route']
    road_segments = []
    for u, v in zip(nodes_int[:-1], nodes_int[1:]):
        # by default, it wll be multi-edges between u and v 
        edgs = G[u][v]
        # If there's only one edge, take it
        if len(edgs) == 1:
            edge_data = list(edgs.values())[0]
        else:
            # For multiple edges, select the one with the minimum 'length' since this matches how we calculated the shortest route earlier
            edge_data = min(edgs.values(), key=lambda x: x.get('length', np.inf))
        # feed (u,v) info into edge_data which didnt contain it by default
        edge_data['u'] = u
        edge_data['v'] = v
        road_segments.append((u, v, edge_data))
        
    edge_data_ls=[]
    free_travel_time_sec_ls=[]
    for segment in road_segments:
        u,v,edge_data=segment
        edge_data_ls.append(edge_data)
        free_travel_time_sec_ls.append(edge_data.get('travel_time', np.nan))
    free_travel_time_total_sec=sum(free_travel_time_sec_ls)
    return (free_travel_time_total_sec,  edge_data_ls)

In [ ]:
# application
data_merged_routes_diff500_chopped[['free_travel_time_total_sec', 
                                                               'edge_data_ls', 
                                                              ]]=data_merged_routes_diff500_chopped.apply(lambda row:
                                                                                                                          pd.Series(extract_road_segments_details(row)),
                                                                                                                          axis=1)

In [ ]:
# add the 'free_travel_time_total_sec' and 'edge_data_ls' columns to data_merged_routes_diff500 and rename it as trips_segs_merge
data_merged_routes_diff500_chopped_subcols=data_merged_routes_diff500_chopped[['free_travel_time_total_sec', 
                                                               'edge_data_ls', 'shortest_route']]
trips_segs_merge=pd.merge(data_merged_routes_diff500, data_merged_routes_diff500_chopped_subcols, on='shortest_route')

### 2.3 Display the road segments in our dataset on OSMnx graph and check their coverage ratio in each region

In [ ]:
# combine all road segments extracted from the existing dataset into a list 
shortest_routes_segs_ls=[]
for x in data_merged_routes_diff500_chopped['edge_data_ls'].values:
    shortest_routes_segs_ls.extend(x)

# use chuncks to deal with memory error
chunk_size = 100000  # Adjust based on available memory
chunks = []
for i in range(0, len(shortest_routes_segs_ls), chunk_size):
    chunk = pd.DataFrame(shortest_routes_segs_ls[i:i + chunk_size])
    chunks.append(chunk)

shortest_routes_segs_df = pd.concat(chunks, ignore_index=True)
# remove list object 
shortest_routes_segs_df=shortest_routes_segs_df.apply(remove_list_obj)
# remove duplicated segments
shortest_routes_segs_df_group=shortest_routes_segs_df.drop_duplicates()

In [ ]:
# Plot the road segments in our dataset on OSMnx graph to check their coverage
fig, ax = plt.subplots(figsize=(12, 12))
# define a function to plot the segments from the calculated shortest routes in our dataset
def plot_shortest_routes(row, graph=G, ax=ax, color='black', linewidth=0.5):
    geometry=row['geometry']
    ax.plot(*geometry.xy, color=color, linewidth=linewidth)
# plot all the edges in Chicago with their region label

region_centroids = edges.groupby('region')['geometry'].apply(lambda x: x.unary_union.centroid)
for region, centroid in region_centroids.items():
    ax.text(centroid.x, centroid.y, f'region{region}',  # Adjust position slightly
            fontsize=6, ha='center', va='center', color='black', 
            bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))

edges.plot(ax=ax, column='region', cmap='tab20', legend=True, linewidth=0.8, legend_kwds={'label': "Region Clusters"})
# Add title
plt.title('Road Segments Coverage Check', fontsize=16)
#apply the defined plot function to the extracted segments df
shortest_routes_segs_df_group.apply(lambda row: plot_shortest_routes(row), axis=1)

### 2.4 Complete the segment level traffic information   

In [ ]:
# define a function to create segment level dataframe while distributing the trip_secs to each road segment proportionally
def create_seg_level_df (row):
    total_free_travel_time=row['free_travel_time_total_sec']
    X=pd.DataFrame(row['edge_data_ls'])[[ 'lanes',  'highway', 
        'speed_kph', 'travel_time', 'region',  'u', 'v', 'junction',
       'bridge', 'tunnel', 'access']]
    X['inferred_travel_time_sec']=np.maximum(X['travel_time'], X['travel_time']/total_free_travel_time*row['trip_seconds'])
    X[['Unnamed: 0', 'trip_start_timestamp', 'trip_end_timestamp', 
       'temp', 'dwpt', 'rhum', 'prcp', 'wdir', 'wpgt', 'pres']]=row[['Unnamed: 0', 'trip_start_timestamp', 'trip_end_timestamp', 
       'temp', 'dwpt', 'rhum', 'prcp', 'wdir', 'wpgt', 'pres']]
    return X

**Note: if you run the following cell in notebook, it probably will cause page snapped (the data size is over 6GB). You can run it in Anaconda prompt instead**

In [ ]:
chunk_size = 100  # Adjust based on available memory
# Open the file in append mode and write chunks incrementally
for i in range(0, len(trips_segs_merge), chunk_size):
    chunk = trips_segs_merge.iloc[i:i + chunk_size]
    chunk_result = pd.concat(chunk.apply(create_seg_level_df, axis=1).to_list(), ignore_index=True)   
    # Write chunk results to file
    chunk_result.to_csv('segment_level_data.csv', mode='a', index=False, header=(i == 0));  

## Chapter 3 Check the Temporal Traffic Pattern Completeness for Each Region 

In [ ]:
# Load the CSV lazily with Dask due to the large size of the dataset
ddf = dd.read_csv('segment_level_data.csv')

In [ ]:
# split the huge dataset into 29 smaller datasets representing 29 regions
for region in range(29):
    print(region)
    filtered_region = ddf[ddf['region'] == region]   
    # Save the filtered data directly to a file
    filtered_region.to_csv(f'filtered_region_{region}.csv', single_file=True, index=False)

In [ ]:
# load the trip level dataset for comparison
data_merged_routes_diff500=pd.read_csv('Chicago_routes_filter.csv')

In [ ]:
# convert the timestamp columns into datetime object and extract year, month, day of week, and hour temporal infornation
data_merged_routes_diff500['trip_start_timestamp']=pd.to_datetime(data_merged_routes_diff500['trip_start_timestamp'])
data_merged_routes_diff500['trip_end_timestamp']=pd.to_datetime(data_merged_routes_diff500['trip_end_timestamp'])
data_merged_routes_diff500['year']=data_merged_routes_diff500['trip_start_timestamp'].dt.year
data_merged_routes_diff500['month']=data_merged_routes_diff500['trip_start_timestamp'].dt.month
data_merged_routes_diff500['dayname']=data_merged_routes_diff500['trip_start_timestamp'].dt.day_name()
data_merged_routes_diff500['hour_start']=data_merged_routes_diff500['trip_start_timestamp'].dt.hour
data_merged_routes_diff500['hour_end']=data_merged_routes_diff500['trip_end_timestamp'].dt.hour
# generate yearly traffic pattern 
total_yearly=data_merged_routes_diff500['year'].value_counts(normalize=True).sort_index()
yearly_comp=pd.DataFrame({'total_yearly': total_yearly})
# generate monthly traffic pattern 
total_monthly=data_merged_routes_diff500['month'].value_counts(normalize=True).sort_index()
monthly_comp=pd.DataFrame({'total_monthly': total_monthly})
# generate day of week traffic pattern 
total_daily=data_merged_routes_diff500['dayname'].value_counts(normalize=True).reindex(['Monday', 'Tuesday','Wednesday', 'Thursday',                                                                                         'Friday', 'Saturday', 'Sunday'], axis=1)
daily_comp=pd.DataFrame({'total_daily': total_daily})
# generate hourly traffic pattern 
total_hourly=data_merged_routes_diff500['hour_start'].value_counts(normalize=True).sort_index()
hourly_comp=pd.DataFrame({'total_hourly': total_hourly})

In [ ]:
# Loop through each region for the same operation as above
for region in range(29):
    filtered_region=pd.read_csv(f'filtered_region_{region}.csv')   
    filtered_region['trip_start_timestamp']=pd.to_datetime(filtered_region['trip_start_timestamp'])
    filtered_region['trip_end_timestamp']=pd.to_datetime(filtered_region['trip_end_timestamp'])
    filtered_region['year']=filtered_region['trip_start_timestamp'].dt.year
    filtered_region['month']=filtered_region['trip_start_timestamp'].dt.month
    filtered_region['dayname']=filtered_region['trip_start_timestamp'].dt.day_name()
    filtered_region['hour_start']=filtered_region['trip_start_timestamp'].dt.hour
    filtered_region['hour_end']=filtered_region['trip_end_timestamp'].dt.hour
    
    # temporal pattern comparison
    region_yearly=filtered_region['year'].value_counts(normalize=True).sort_index()
    yearly_comp[f'region_{region}_yearly']=region_yearly
    region_monthly=filtered_region['month'].value_counts(normalize=True).sort_index()
    monthly_comp[f'region_{region}_monthly']=region_monthly
    region_daily=filtered_region['dayname'].value_counts(normalize=True).reindex(['Monday', 'Tuesday','Wednesday', 'Thursday', 
                                                                                  'Friday', 'Saturday', 'Sunday'], axis=1)
    daily_comp[f'region_{region}_daily']=region_daily
    region_hourly=filtered_region['hour_start'].value_counts(normalize=True).sort_index()
    hourly_comp[f'region_{region}_hourly']=region_hourly 
    # Clear memory by deleting the current region's dataframe
    del filtered_region
    # Force garbage collection
    gc.collect()
    print(f"Processed and cleared region {region}")

In [ ]:
monthly_comp.plot(kind='line')
plt.ylabel('normalized traffic amount')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))  # Position legend outside the plot

In [ ]:
daily_comp.plot(kind='line')
plt.ylabel('normalized traffic amount')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))

In [ ]:
hourly_comp.loc[:, 'region_0_hourly':].plot(kind='line')
hourly_comp.loc[:, 'total_hourly'].plot(kind='line', color='black')
plt.ylabel('normalized traffic amount')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))

Analysis:
- we can see each region follows a similar temporal traffic patten to the overall dataset by monthly, daily, and hourly, which means all regions retain complete temporal traffic information.

## Chapter 4 Road Segments Clustering within Each Region

### 4.1 Data preparation for KMeans clustering

In [ ]:
#timeframe here represents morning, afternoon, and night timeframes
regions_timeframe={}
regions_hourly={}
# Loop through each region
for region in range(29):
    filtered_region=pd.read_csv(f'filtered_region_{region}.csv')   
    filtered_region['trip_start_timestamp']=pd.to_datetime(filtered_region['trip_start_timestamp'])
    filtered_region['trip_end_timestamp']=pd.to_datetime(filtered_region['trip_end_timestamp'])
    filtered_region['year']=filtered_region['trip_start_timestamp'].dt.year
    filtered_region['month']=filtered_region['trip_start_timestamp'].dt.month
    filtered_region['dayname']=filtered_region['trip_start_timestamp'].dt.day_name()
    filtered_region['hour_start']=filtered_region['trip_start_timestamp'].dt.hour
    filtered_region['hour_end']=filtered_region['trip_end_timestamp'].dt.hour
    filtered_region['inferred_travel_time_sec'] = np.maximum(filtered_region['inferred_travel_time_sec'], filtered_region['travel_time'])
# add a new column 'morning_afternoon_night' to group hours of day to morning, afternoon, and night time frames 
    filtered_region['morning_afternoon_night'] = filtered_region['hour_start'].apply(
    lambda x: 'morning' if 6 <= x < 12 
              else 'afternoon' if 12 <= x <18 
              else 'night' )
   # aggregate the inferred_travel_time_sec by each timeframe for each segment 
    filtered_region_timeframe=filtered_region.groupby(
    ['u', 'v', 'morning_afternoon_night']
    )['inferred_travel_time_sec'].median().unstack().reindex(['morning', 'afternoon', 'night'], axis=1)
    regions_timeframe[f'filtered_region_{region}_timeframe']=filtered_region_timeframe
    # aggregate the inferred_travel_time_sec by hourly for each segment 
    filtered_region_hourly=filtered_region.groupby(
    ['u', 'v', 'hour_start']
    )['inferred_travel_time_sec'].median().unstack()
    regions_hourly[f'filtered_region_{region}_hourly']=filtered_region_hourly
    
    # Clear memory by deleting the current region's dataframe
    del filtered_region
    # Force garbage collection
    gc.collect()
    print(f"Processed and cleared region {region}")

In [ ]:
# define a function to fill NaN values in the hourly traffic distribution for each segment:
## to fill each hourly missing data by the corresponding morning or afternoon or night aggregated traffic data for each segment 
## if the morning or afternoon or night aggregated traffic data is also missing, fill it by using the average over the existing other timeframes.
morning_hours=[6,7,8,9,10,11]
afternoon_hours=[12,13,14,15,16,17]
night_hours=[18,19,20,21,22,23,0,1,2,3,4,5]
def fill_na(row):
    for hour_col in range(24):
        if pd.isna(row[hour_col]):
            if hour_col in morning_hours:
                if not pd.isna(row['morning']):
                    row[hour_col]=row['morning']
                else:
                    row[hour_col]=np.nanmean([row['afternoon'], row['night']])
            elif hour_col in afternoon_hours:
                if not pd.isna(row['afternoon']):
                    row[hour_col]=row['afternoon']
                else:
                    row[hour_col]=np.nanmean([row['morning'], row['night']])
            else:
                if not pd.isna(row['night']):
                    row[hour_col]=row['night']
                else:
                    row[hour_col]=np.nanmean([row['morning'], row['afternoon']])
    return row

In [ ]:
#application
regions_hourly_fillna={}
for region in range(29):
    filtered_region_merge=pd.merge(regions_hourly[f'filtered_region_{region}_hourly'],
                                              regions_timeframe[f'filtered_region_{region}_timeframe'], left_index=True, right_index=True)
    regions_hourly_fillna[f'filtered_region_{region}_hourly_fillna']=filtered_region_merge.apply(fill_na, axis=1).loc[:, :23]    

### 4.2 Use elbow method to determine K for each region

In [ ]:
region=0
X=regions_hourly_fillna[f'filtered_region_{region}_hourly_fillna']
# Extract hourly travel time data
hourly_data = X.values

# Normalize each segment by its total travel time to capture patterns
pattern_data = hourly_data / np.sum(hourly_data, axis=1, keepdims=True)

# Add the sum (scale) back as a feature
scale_feature = np.sum(hourly_data, axis=1).reshape(-1, 1)

# Combine pattern and scale features
combined_features = np.hstack((pattern_data, scale_feature))

In [ ]:
# plot the elbow plot
model = KMeans()
plot_elbow_curve(model, combined_features, cluster_ranges=range(1, min(20, len(X))), figsize=(12, 8))

Analysis:
- from the above plot, we can start with the K that corresponds to the last elbow. In the above example where region=0, K=6.

In [ ]:
# use KMeans to do clustering
kmeans = KMeans(n_clusters=6, random_state=42) 
X['cluster']=kmeans.fit_predict(combined_features) 
X['cluster'].value_counts()

In [ ]:
# Clear memory by deleting the old region's dataframe before creating the new region in the following cell
del filtered_region
del merge
# Force garbage collection
gc.collect()

In [ ]:
# load a new region data
filtered_region=pd.read_csv(f'filtered_region_{region}.csv')   
filtered_region['trip_start_timestamp']=pd.to_datetime(filtered_region['trip_start_timestamp'])
filtered_region['trip_end_timestamp']=pd.to_datetime(filtered_region['trip_end_timestamp'])
filtered_region['year']=filtered_region['trip_start_timestamp'].dt.year
filtered_region['month']=filtered_region['trip_start_timestamp'].dt.month
filtered_region['dayname']=filtered_region['trip_start_timestamp'].dt.day_name()
filtered_region['hour_start']=filtered_region['trip_start_timestamp'].dt.hour
filtered_region['hour_end']=filtered_region['trip_end_timestamp'].dt.hour
filtered_region['inferred_travel_time_sec'] = np.maximum(filtered_region['inferred_travel_time_sec'], filtered_region['travel_time'])
filtered_region=filtered_region.set_index(['u', 'v'])

In [ ]:
# merge the cluster column to the loaded region data
merge=pd.merge(filtered_region, X[['cluster']], left_index=True, right_index=True)
merge['cluster'].value_counts()

In [ ]:
# plot the hourly aggregated inferred_travel_time_sec for each cluster 
time='hour_start'
df_plt=merge.groupby(['cluster', time])['inferred_travel_time_sec'].median().unstack().T
df_plt['total']=merge.groupby(time)['inferred_travel_time_sec'].median()
df_plt.iloc[:,:-1].plot(marker='o')
df_plt['total'].plot(marker='*', color='black', label='total')
plt.ylabel('aggregated_inferred_travel_time_sec')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))

In [ ]:
# plot the hourly traffic pattern for each cluster
df_plt2=merge.groupby('cluster')[time].value_counts(normalize=True).unstack().T
df_plt2['total']=merge[time].value_counts(normalize=True)
df_plt2.iloc[:,:-1].plot(marker='*')
df_plt2['total'].plot(marker='+', color='black', label='total')
plt.ylabel('traffic_amount_ratio')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))

Analysis:
- combine with the above two plots, we can determine whether to adjust K further. For example, if we noticed there exist clusters close to each other in the first plot (the inferred_travel_time_sec hourly distribution), we may want to reduce K, and if we noticed there exist incomplete lines for some clusters in both plots, we can also consider reducing K to try to merge these outliers with other clusters if reasonably. Our objective is to see  well separated complete lines in the first plot and high overlapped complete lines in the second plot. 

Cluster Result: based on checking the clustering result for each of 29 regions one by one, I got the following result. There exist some outliers I didnt merge into other clusters (marked by red) since I didnt want to sacrifice the clustering accuracy because of them. I will figure out a way for them in the following model part.

---

###### Region 0: 6 Clusters
- 0:    96,449 rows
- 4:   54,403
- 3:    50,219
- 2:    37,644
- 1:     3,517
- 5:     1,402

---

###### Region 1: 5 Clusters

- 2:    332,363
- 0:    161,706
- 4:     80,164
- 1:     76,591
- 3:      7,188
---

###### Region 2: 4 Clusters
- 1:    3,380,709
- 3:    2,380,233
- 0:    1,303,532
- 2:      88,891
---

###### Region 3: 5 Clusters
- 1:    401,577
- 3:    209,233
- 2:    154,352
- 0:     89,029
- 4:      7,300
---

###### Region 4: 5 Clusters
- 0    700,478
- 2    532,191
- 1    281,492
- 4     43,570
- 3      5,098
---

###### Region 5: 5 Clusters
- 0    37,532
- 4     7,376
- 2     7,071
- 3     6,707
- 1     5,446


---

###### Region 6: 5 Clusters

- 4    440,870
- 2    270,005
- 0    146,883
- 1    117,562
- 3     10,666

---

###### Region 7: 5 Clusters

- 4    36,195
- 0    34,550
- 3    18,996
- 2    12,045
- 1     3,185


---

###### Region 8: 4 Clusters
- 2    372,019
- 0    221,825
- 1    100,684
- 3     36,581



---

###### Region 9: 5 Clusters
- 1    233,719
- 0     62,952
- 2     42,501
- 4     26,054
- 3      4,746


---

###### Region 10: 5 Clusters
- 1    44,793
- 0    26,938
- 3    11,885
- 2     2,402
- 4     **<span style="color:red;">13</span>**
---

###### Region 11: 5 Clusters
- 2    40,320
- 0    11,656
- 3     1,609
- 1       **<span style="color:red;">125</span>**  
- 4       **<span style="color:red;">71</span>**  
---

###### Region 12: 5 Clusters

- 0    116,669
- 4     48,947
- 2     37,257
- 1       518
- 3     **<span style="color:red;">3</span>**

---

###### Region 13: 5 Clusters

- 2    6,148,668
- 1    3,088,207
- 0    3,056,195
- 3     619,525
- 4      98,524

---

###### Region 14: 6 Clusters

- 5    1,157,573
- 0     765,385
- 4     525,156
- 3     520,131
- 1     287,293
- 2      11,593

---

###### Region 15: 5 Clusters
- 4    137,890
- 1     99,674
- 0     40,213
- 3      7,868
- 2      **<span style="color:red;">8</span>**

---

###### Region 16: 5 Clusters
- 0    35,594
- 3    19,156
- 4    10,396
- 1     4,800
- 2     2,892

---

###### Region 17: 5 Clusters

- 3    451983
- 1    379076
- 0    351754
- 2    183072
- 4     20732


---

###### Region 18: 6 Clusters

- 5    128354
- 0    100929
- 2     85153
- 1     78779
- 4     35143
- 3      1168

---

###### Region 19: 4 Clusters

- 2    139,630
- 0     94,054
- 3     28,192
- 1      3,220


---

###### Region 20: 5 Clusters
- 4    180024
- 0     70587
- 1     43093
- 2      2896
- 3      **<span style="color:red;">1</span>**

---

###### Region 21: 5 Clusters
- 0    1,289,560
- 4    1,005,728
- 2     450,651
- 1     237,195
- 3      38,688

---

###### Region 22: 5 Clusters 
- 2    218,782
- 0    105,606
- 1     80,935
- 4     17,964
- 3      2,342
---

###### Region 23: 4 Clusters
- 3    12658
- 1     8365
- 0     5909
- 2    **<span style="color:red;">425</span>**  


---

###### Region 24: 5 Clusters
- 0    304,168
- 4    170,309
- 3     59,218
- 2      4,870
- 1      2,257

---

###### Region 25: 5 Clusters
- 1    1,045,304
- 3     546,643
- 2     412,322
- 0     166,239
- 4      11,376



---

###### Region 26: 6 Clusters 
- 1    542,353
- 5    176,423
- 0    108,009
- 4     78,770
- 3     16,192
- 2      **<span style="color:red;">6</span>**

---

###### Region 27: 5 Clusters
- 1    398,144
- 3    200,025
- 0    154,587
- 2     70,160
- 4       961

---

###### Region 28: 4 Clusters
- 2    570
- 0    223
- 1     52
- 3     20

### 4.3 Save the obtained cluster information for each region to the segment level dataframe

In [ ]:
cluster_result=[6,5,4,5,5,5,5,5,4,5,5,5,5,5,6,5,5,5,6,4,5,5,5,4,5,5,6,5,4]
time='hour_start'
regions_traveltime={}
regions_traffic={}
for region in range(29):
    X=regions_hourly_fillna[f'filtered_region_{region}_hourly_fillna']
    # Extract hourly travel time data
    hourly_data=X.values
    # Normalize each road segment by its total travel time to capture patterns
    pattern_data=hourly_data/np.sum(hourly_data, axis=1, keepdims=True)
    # Add the sum (scale) back as a feature
    scale_feature=np.sum(hourly_data, axis=1).reshape(-1,1)
    # Combine pattern and scale features
    combined_features=np.hstack((pattern_data, scale_feature))
    kmeans=KMeans(n_clusters=cluster_result[region], random_state=42)
    X['cluster']=kmeans.fit_predict(combined_features)
    filtered_region=pd.read_csv(f'filtered_region_{region}.csv')
    filtered_region['trip_start_timestamp']=pd.to_datetime(filtered_region['trip_start_timestamp'])
    filtered_region['trip_end_timestamp']=pd.to_datetime(filtered_region['trip_end_timestamp'])
    filtered_region['year']=filtered_region['trip_start_timestamp'].dt.year
    filtered_region['month']=filtered_region['trip_start_timestamp'].dt.month
    filtered_region['dayname']=filtered_region['trip_start_timestamp'].dt.day_name()
    filtered_region['hour_start']=filtered_region['trip_start_timestamp'].dt.hour
    filtered_region['hour_end']=filtered_region['trip_end_timestamp'].dt.hour
    filtered_region['inferred_travel_time_sec'] = np.maximum(filtered_region['inferred_travel_time_sec'], filtered_region['travel_time'])
    filtered_region=filtered_region.set_index(['u', 'v'])
    merge=pd.merge(filtered_region, X[['cluster']], left_index=True, right_index=True)
    df_traveltime=merge.groupby(['cluster', time])['inferred_travel_time_sec'].median().unstack().T
    df_traveltime['total']=merge.groupby(time)['inferred_travel_time_sec'].median()
    regions_traveltime[f'region_{region}']=df_traveltime
    df_traffic=merge.groupby('cluster')[time].value_counts(normalize=True).unstack().T
    df_traffic['total']=merge[time].value_counts(normalize=True)
    regions_traffic[f'region_{region}']=df_traffic
    merge.to_csv(f'region_{region}_withcluster.csv', index=True)
    # Clear memory by deleting the current region's dataframe
    del filtered_region
    del merge
    # Force garbage collection
    gc.collect()
    print(f"Processed and cleared region {region}")

In [ ]:
# compare the inferred travel time hourly distribution for each region with the total dataset
fig, axs=plt.subplots(10, 3, figsize=(20,60))
for region in range(29):
    row_ind=region//3
    col_ind=region%3
    ax=axs[row_ind, col_ind]
    regions_traveltime[f'region_{region}'].iloc[:,:-1].plot(marker='o', ax=ax)
    regions_traveltime[f'region_{region}']['total'].plot(marker='*', color='black', label='total', ax=ax)
    ax.set_title(f'Region {region}', fontsize=10)
    ax.set_ylabel('aggregated_inferred_travel_time_sec', fontsize=8)
    ax.legend(fontsize=8)
# Hide unused subplot
axs[9, 2].set_visible(False)

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# compare the hourly traffic pattern  for each region with the total dataset
fig, axs=plt.subplots(10, 3, figsize=(20,60))
for region in range(29):
    row_ind=region//3
    col_ind=region%3
    ax=axs[row_ind, col_ind]
    regions_traffic[f'region_{region}'].iloc[:,:-1].plot(marker='*', ax=ax)
    regions_traffic[f'region_{region}']['total'].plot(marker='+', color='black', label='total', ax=ax)
    ax.set_title(f'Region {region}', fontsize=10)
    ax.set_ylabel('normalized traffic amount', fontsize=8)
    ax.legend(fontsize=8)
# Hide unused subplot
axs[9, 2].set_visible(False)

# Adjust layout
plt.tight_layout()
plt.show()

### 4.4 Display the above plotted results onto map

#### 4.4.1 Region data preparation

In [ ]:
# select a region dataset to display
region=1
region_withcluster=pd.read_csv(f'region_{region}_withcluster.csv')
region_withcluster.head()

In [ ]:
region_data=region_withcluster.groupby(['cluster', 'u', 'v', 'hour_start'])[['inferred_travel_time_sec', 'travel_time']].median()
region_data.head()

In [ ]:
# Add a congestion level column by comparing inferred_travel_time_sec with travel_time
def calculate_congestion_level(row):
    if pd.isna(row['inferred_travel_time_sec']):
        ratio==1
        return (ratio, 'lightgreen', 'free-flow')  # Mark NaN values as free flow
    ratio = row['inferred_travel_time_sec'] / row['travel_time']
    if ratio <= 1.2:
        return (ratio, 'lightgreen', 'free_flow')  # Free flow
    elif ratio <= 1.5:
        return (ratio, 'green', 'slight congestion')  # Slight congestion
    elif ratio <= 2.0:
        return (ratio, '#E65100', 'medium congestion')  # Medium congestion
    elif ratio >2:
        return (ratio, 'darkred', 'severe congestion')  # Severe congestion
#application
region_data[['congestion_ratio', 'congestion_level', 'congestion_level_explanation']] = region_data.apply(lambda row: pd.Series(
                                                                                                                                        calculate_congestion_level(row)), axis=1)

In [ ]:
# combine 'highway' column to region_data to reflect the road type information on map
m1=region_data.reset_index()
m2=region_withcluster.drop_duplicates(['u','v'])[['u', 'v', 'highway']]
merge=pd.merge(m1, m2, on=['u', 'v'])
region_data=merge.set_index(['cluster', 'u', 'v','hour_start'])

#### 4.4.2 Plot road segments on map, colored by congestion level

In [ ]:
#create an OSMnx graph for Chicago city
place = 'Chicago, Illinois, USA'
G = ox.graph_from_place(place, network_type='drive')

#add speed limit and free flow travel_time attributes to the edges.
G = ox.add_edge_speeds(G)
G = ox.add_edge_travel_times(G)
# create geo-dataframe for both nodes and edges 
nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)
# add geometry from edges to G for plot in the following
for idx, row in edges.reset_index().iterrows():
    u, v, key = row['u'], row['v'], row['key']
    G[u][v][key]['geometry'] = row['geometry']

In [ ]:
# Define a function to plot road segments of a region with cluster and hour-based filtering on map
def plot_region_with_filters(region_data, region_id, G):
    # Initialize map centered on Chicago
    m = folium.Map(location=[41.8781, -87.6298], zoom_start=11)

    # Add a base layer with no visible data
    folium.TileLayer("cartodbdark_matter", name="No Data Visible", control=True).add_to(m)

    # Iterate through clusters and hours
    clusters = region_data.index.get_level_values("cluster").unique()

    for cluster in clusters:
        # Filter data for the current cluster 
        cluster_data = region_data[region_data.index.get_level_values("cluster") == cluster]
        for hour in range(24):  # Loop through hours from 0 to 23
            hour_data = cluster_data[cluster_data.index.get_level_values("hour_start") == hour]    
            # Prepare GeoJSON data
            geojson_data = {"type": "FeatureCollection", "features": []}

            for index, row in hour_data.iterrows():
                _, u, v,  hour_start = index
                    
                # Try to get the edge geometry
                edgs = G[u][v]
                if len(edgs) == 1:
                    edge_data = list(edgs.values())[0]
                else:
                    edge_data = min(edgs.values(), key=lambda x: x.get("length", float("inf")))

                # Extract coordinates from LineString
                line_coords = list(edge_data["geometry"].coords)

                # Add road segment as a GeoJSON feature
                feature = {
                        "type": "Feature",
                        "geometry": {
                            "type": "LineString",
                            "coordinates": line_coords
                        },
                        "properties": {
                            "congestion_level": row["congestion_level"],
                            "popup": (
                                f"Inferred Travel Time: {row['inferred_travel_time_sec']} sec<br>"
                                f"Free Flow Time: {row['travel_time']} sec<br>"
                                f"Congestion: {row['congestion_level_explanation']}<br>"
                                f"Congestion_ratio: {row['congestion_ratio']}<br>"
                                f"roadtype: {row['highway']}"
                            )
                        }
                    }
                geojson_data["features"].append(feature)

          
                # Add GeoJson layer for the specific cluster and hour, hidden by default
            folium.GeoJson(
                    geojson_data,
                    name=f"Cluster {cluster}  | Hour {hour}",  # Layer name for toggling
                    style_function=lambda feature: {
                            "color": feature["properties"]["congestion_level"],
                            "weight": 5,
                            "opacity": 1.0
                        },
                        popup=folium.GeoJsonPopup(
                            fields=["popup"],
                            aliases=["Info: "],
                            sticky=True,
                            max_width=400
                        ),
                        show=False  # Hide this layer by default
                    ).add_to(m)

    # Add layer control to toggle between combinations
    folium.LayerControl(collapsed=False).add_to(m)

    # Add title
    title_html = f"""
        <h3 align="center" style="font-size:16px"><b>Region {region_id} Traffic Congestion<br></h3>
        """
    m.get_root().html.add_child(folium.Element(title_html))

    return m


# Example usage
map_region = plot_region_with_filters(region_data=region_data, region_id=1, G=G)
map_region.save("region_1_traffic_filters_map_new.html")  # Save as an interactive HTML
map_region

#### 4.4.3 Display the shortest routes calculated by inferred travel time and free flow travel time on map for comparison

In [ ]:
# randomly pick an hour time
hour_time=10
region_data_hour=region_data[region_data.index.get_level_values('hour_start')==hour_time]

In [ ]:
#define a function to create a directed graph from the region_data dataframe.
def create_subset_graph_with_inferred_time(G, region_data_hour):
    # Extract relevant edges (u, v) from region_data_hour
    relevant_edges = [(row.name[1], row.name[2]) for _, row in region_data_hour.iterrows()]
    relevant_nodes = set([node for edge in relevant_edges for node in edge])

    # Handle multi-edge graph by selecting the shortest edge for each (u, v)
    subset_edges = []
    for u, v in relevant_edges:
        if G.has_edge(u, v):  
            # Find the edge with the minimum 'length' 
            min_edge_key = min(G[u][v], key=lambda k: G[u][v][k].get('length', float('inf')))
            subset_edges.append((u, v, min_edge_key))

    # Create the subset graph with the selected edges
    G_c = G.edge_subgraph(subset_edges).copy()

    # Add the inferred_travel_time_sec attribute from region_data_hour to the edges in G_c
    for _, row in region_data_hour.iterrows():
        u, v = row.name[1], row.name[2]
        edge_keys = list(G_c[u][v].keys())  
        first_key = edge_keys[0]  # Get the first key
        G_c[u][v][first_key]['inferred_travel_time_sec'] = row['inferred_travel_time_sec']
        G_c[u][v][first_key]['congestion_level_explanation'] = row['congestion_level_explanation']
        G_c[u][v][first_key]['congestion_ratio'] = row['congestion_ratio']


    # Ensure all relevant nodes are included
    for node in relevant_nodes:
        if node not in G_c.nodes and node in G:
            G_c.add_node(node, **G.nodes[node])

    return G_c


# Example usage
G_c = create_subset_graph_with_inferred_time(G, region_data_hour)
print(f"Subset graph has {G_c.number_of_nodes()} nodes and {G_c.number_of_edges()} edges.")

# Example to inspect an edge's attributes
u, v, key = next(iter(G_c.edges))
print(f"Edge ({u}, {v}) attributes: {G_c[u][v][key]}")

In [ ]:
# define a function to find a pair of nodes (start_node, end_node) in the graph with multiple routes between them
def find_pair_with_multiple_routes(region_data_hour, G_c):
    
    # Get unique list of nodes
    unique_nodes = list(set(G_c.nodes))  

    # Find a pair of nodes with multiple paths
    for i, start_node in enumerate(unique_nodes):
        for end_node in unique_nodes[i + 1:]:  # Check unique pairs (start_node, end_node)
            paths = list(nx.all_simple_paths(G_c, source=start_node, target=end_node, cutoff=5))  # Limit to 5 edges for simplicity
            if len(paths) > 1:  # Check if there are multiple paths
                return start_node, end_node
    # If no suitable pair is found, return None
    return None, None
# Example usage
start_node, end_node = find_pair_with_multiple_routes(region_data_hour, G_c)
if start_node and end_node:
    print(f"Found suitable nodes: Start Node = {start_node}, End Node = {end_node}")
else:
    print("No suitable nodes with multiple routes found.")


In [ ]:
# define a function to calculate the shortest route in a graph based on a given weight
def calculate_shortest_route(G_c, start_node, end_node, weight):

    # Calculate the shortest path
    path = nx.shortest_path(G_c, source=start_node, target=end_node, weight=weight)
    
    # Calculate the total weight of the path
    total_time = nx.shortest_path_length(G_c, source=start_node, target=end_node, weight=weight)
    
    return path, total_time


# Shortest route based on real traffic time
path_inferred, time_inferred = calculate_shortest_route(G_c, start_node, end_node, weight='inferred_travel_time_sec')
print(f"Shortest path (real traffic): {path_inferred}, Total time: {time_inferred} sec")

# Shortest route based on free-flow travel time
path_free_flow, time_free_flow = calculate_shortest_route(G_c, start_node, end_node, weight='travel_time')
print(f"Shortest path (free flow): {path_free_flow}, Total time: {time_free_flow} sec")

In [ ]:
# Define a function to display the calculated shortest routes on a map with detailed segment traffic info
def display_shortest_routes_with_details(G_c, start_node, end_node):

    # Initialize map centered at the start node
    start_coords = (G_c.nodes[start_node]['y'], G_c.nodes[start_node]['x'])  # (latitude, longitude)
    m = folium.Map(location=start_coords, zoom_start=13)

    # Define weights and their visualization styles
    route_styles = {
        'inferred_travel_time_sec': {'color': '#E65100', 'dash_array': None},  # Solid red line
        'travel_time': {'color': '#E65100', 'dash_array': '5,10'},  # Dashed green line
    }

    for weight, style in route_styles.items():
        # Calculate shortest path and its nodes
        path = nx.shortest_path(G_c, source=start_node, target=end_node, weight=weight)
        
        # Extract edges and their corresponding traffic information
        for u, v in zip(path[:-1], path[1:]):  # Iterate over consecutive nodes in the path
            edge_data = G_c[u][v]  # Access edge data
            edge_keys = list(G_c[u][v].keys())
            first_key = edge_keys[0]  # get the first key
            edge_geometry = edge_data[first_key].get("geometry")

        
            # Extract traffic information
            inferred_time = edge_data[first_key].get("inferred_travel_time_sec", "N/A")
            free_flow_time = edge_data[first_key].get("travel_time", "N/A")
            congestion_level=edge_data[first_key].get("congestion_level_explanation", "N/A")
            congestion_ratio=edge_data[first_key].get("congestion_ratio", "N/A")

            # Add the edge as a polyline to the map
            line_coords = list(edge_geometry.coords)
            line_coords_reversed = [[lat, lon] for lon, lat in line_coords]  # Reverse for folium
            folium.PolyLine(
                locations=line_coords_reversed,
                color=style['color'],
                weight=5,
                dash_array=style['dash_array'],  # Add dash style for travel_time route
                tooltip=f"<b>Route by {weight}</b><br>"
                        f"Start: {u}, End: {v}<br>"
                        f"Inferred Time: {inferred_time} sec<br>"
                        f"Free Flow Time: {free_flow_time} sec<br> "
                        f"Congestion_level: {congestion_level}<br>"
                        f"Congestion_ratio: {congestion_ratio}<br>"
            ).add_to(m)

    # Add markers for start and end nodes
    folium.Marker(location=start_coords, popup="Start Node", icon=folium.Icon(color="blue")).add_to(m)
    end_coords = (G_c.nodes[end_node]['y'], G_c.nodes[end_node]['x'])
    folium.Marker(location=end_coords, popup="End Node", icon=folium.Icon(color="blue")).add_to(m)
     # Add title
    title_html = f"""
        <h3 align="center" style="font-size:16px"><b>Region {region} Traffic Congestion at 10 am<br></h3>
        """
    m.get_root().html.add_child(folium.Element(title_html))


    return m


# Example usage
map_routes = display_shortest_routes_with_details(G_c, start_node, end_node)
map_routes.save("shortest_routes_with_details_map_new.html")  # Save as an interactive HTML
map_routes

## Chapter 5  Build ML models for predicting the  travel time for each road segment cluster

### 5.1 Check the clustering accuracy by plotting the inferred travel time histogram for each cluster

In [ ]:
# pick one region
region=1
region_withcluster=pd.read_csv(f'region_{region}_withcluster.csv')
region_withcluster.head()

In [ ]:
region_withcluster.columns

In [ ]:
# check the data amount within each cluster
region_withcluster['cluster'].value_counts()

In [ ]:
# pick one cluster for specific analysis
cluster=0
filtered_cluster=region_withcluster[region_withcluster['cluster']==cluster]
filtered_cluster.shape

In [ ]:
# check the duplicated trip_start_timestamps within a cluster
duplicated_time=filtered_cluster.duplicated('trip_start_timestamp').sum()
# calculate the valid number of rows (unique timestamps) we can use for the modeling
unique_timestamps=filtered_cluster.shape[0]-duplicated_time
unique_timestamps

In [ ]:
# check the clustering accuracy by plotting the inferred_travel_time_sec histogram for each hour, expecting to see a bell shaped distribution 
fig, axs=plt.subplots(6, 4, figsize=(30,30))
for hour in range(24):
    row_ind=hour//4
    col_ind=hour%4
    ax=axs[row_ind, col_ind]
    filtered_cluster_hour=filtered_cluster[filtered_cluster['hour_start']==hour]
    filtered_cluster_hour['inferred_travel_time_sec'].plot(ax=ax, kind='hist', bins=100)
    ax.set_title(f'Cluster {cluster}', fontsize=10)
    ax.set_ylabel('inferred_travel_time_sec_frequency', fontsize=8)
# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
#check the overal histogram for all hours
filtered_cluster['inferred_travel_time_sec'].plot( kind='hist', bins=100)

In [ ]:
filtered_cluster['inferred_travel_time_sec'].describe().drop('count')

Analysis:
- for the hourly histograms, if all of them comply with a bell shape, we can say the clustering works well by grouping road segments with good travel time similarity.

**if no further clustering is needed, check the accuracy  about aggregating the inferred_travel_time_sec by grouping duplicated timestamp**

In [ ]:
# check the duplicated trip_start_timestamp 
filtered_cluster['trip_start_timestamp'].value_counts()

In [ ]:
#randomly pick a timestamp sample to check the accuracy
sample_time='2023-11-11 18:45:00' 
filtered_cluster[filtered_cluster['trip_start_timestamp']==sample_time]['inferred_travel_time_sec'].plot(kind='hist', bins=50)

In [ ]:
med=filtered_cluster[filtered_cluster['trip_start_timestamp']==sample_time]['inferred_travel_time_sec'].median()
avg=filtered_cluster[filtered_cluster['trip_start_timestamp']==sample_time]['inferred_travel_time_sec'].median()
print(f'median: {med}, mean: {avg}')

Analysis:
- from the above histogram, we can see the variance of the inferred travel time with the same timestamps is not significant.
- the mean and medain are close to each other, and each of them can be a good representative for the group.

In [ ]:
# convert the trip_start_timestamp column to datetime format
filtered_cluster['trip_start_timestamp'] = pd.to_datetime(filtered_cluster['trip_start_timestamp'])
# add a new column that extracts the day of a month for the following analysis
filtered_cluster['day']=filtered_cluster['trip_start_timestamp'].dt.day

In [ ]:
filtered_cluster.columns

In [ ]:
# aggregate the rows with the same timestamp
filtered_cluster_groupbytime=pd.DataFrame({'inferred_traveltime_sec_updated': 
                                                 filtered_cluster.groupby('trip_start_timestamp')['inferred_travel_time_sec'].median()})
# Set the trip_start_timestamp column as the index
filtered_cluster.set_index('trip_start_timestamp', inplace=True)
# Sort the DataFrame by the index in ascending order
filtered_cluster.sort_index(inplace=True)
# update the filtered_cluster dataframe after groupby timestamp so as to remove the columns which are specific to individual road segment since 
## the aggregation operation made these columns meaningless
filtered_cluster_updatedbytime=pd.merge(filtered_cluster_groupbytime, filtered_cluster.loc[:, 
                                                            ['temp', 'dwpt', 'rhum', 'prcp', 'wdir',  'pres', 'year', 'month','dayname', 'day', 'hour_start']].copy(),
                                                            left_index=True, right_index=True)
# drop the rows with duplicated timestamp
filtered_cluster_updatedbytime=filtered_cluster_updatedbytime[~filtered_cluster_updatedbytime.index.duplicated(keep='first')]
filtered_cluster_updatedbytime

In [ ]:
# aggregate rows with the same hour time of the same day since we want to focus on hourly data 
filtered_cluster_groupbyhour=filtered_cluster_updatedbytime.groupby(['year', 'month', 'day', 'hour_start'])[['inferred_traveltime_sec_updated', 'temp', 'dwpt', 'rhum', 'prcp', 'wdir', 'pres']].median()
filtered_cluster_groupbyhour=filtered_cluster_groupbyhour.reset_index()
filtered_cluster_updatedbyhour=pd.merge(filtered_cluster_updatedbytime.loc[:, 
                                                            [ 'year', 'month','day', 'hour_start', 'dayname']].reset_index(), filtered_cluster_groupbyhour,
                                                            on=['year', 'month', 'day', 'hour_start'])
filtered_cluster_updatedbyhour=filtered_cluster_updatedbyhour.set_index('trip_start_timestamp').drop_duplicates()
filtered_cluster_updatedbyhour

**if needs to further divide the cluster into more inner clusters**

In [ ]:
# add a new column 'morning_afternoon_night' to group hours of day to morning, afternoon, and night time frames 
filtered_cluster['morning_afternoon_night'] = filtered_cluster['hour_start'].apply(
lambda x: 'morning' if 6 <= x < 12 
          else 'afternoon' if 12 <= x <18 
          else 'night' )
# fill the na in hourly aggregation for each segment by their corresponding timeframe aggregated values 
filtered_cluster_timeframe=filtered_cluster.groupby(['u', 'v', 'morning_afternoon_night'])['inferred_travel_time_sec'].median().unstack()
filtered_cluster_hourly=filtered_cluster.groupby(['u', 'v', 'hour_start'])['inferred_travel_time_sec'].median().unstack()
filtered_cluster_merge=pd.merge(filtered_cluster_hourly, filtered_cluster_timeframe, left_index=True, right_index=True)
filtered_cluster_merge_fillna=filtered_cluster_merge.apply(fill_na, axis=1)

In [ ]:
# Extract hourly travel time data
hourly_data = filtered_cluster_merge_fillna.loc[:, 0:23].values
# Normalize each road type by its total travel time to capture patterns
pattern_data = hourly_data / np.sum(hourly_data, axis=1, keepdims=True)
# Add the sum (scale) back as a feature
scale_feature = np.sum(hourly_data, axis=1).reshape(-1, 1)
# Combine pattern and scale features
combined_features = np.hstack((pattern_data, scale_feature))

In [ ]:
#### plot the elbow plot
model = KMeans()
plot_elbow_curve(model, combined_features, cluster_ranges=range(1, min(10, combined_features.shape[0] + 1)), figsize=(12, 8))

In [ ]:
# run KMeans for further clustering
kmeans = KMeans(n_clusters=4, random_state=42) 
filtered_cluster_merge_fillna['inner_cluster']=kmeans.fit_predict(combined_features) 
filtered_cluster_merge_fillna['inner_cluster'].value_counts()

In [ ]:
# merge the inner cluster information with the original region dataset
filtered_cluster=region_withcluster[region_withcluster['cluster']==cluster]
filtered_cluster=pd.merge(filtered_cluster, filtered_cluster_merge_fillna[['inner_cluster']], left_on=['u', 'v'], right_index=True)

In [ ]:
# check the inner clustering accuracy again
inner_cluster=1
filtered_inner_cluster=filtered_cluster[filtered_cluster['inner_cluster']==inner_cluster]
fig, axs=plt.subplots(6, 4, figsize=(30,30))
for hour in range(24):
    row_ind=hour//4
    col_ind=hour%4
    ax=axs[row_ind, col_ind]
    filtered_inner_cluster_hour=filtered_inner_cluster[filtered_inner_cluster['hour_start']==hour]
    filtered_inner_cluster_hour['inferred_travel_time_sec'].plot(ax=ax, kind='hist', bins=100)
    ax.set_title(f'inner_cluster {inner_cluster}', fontsize=10)
    ax.set_ylabel('inferred_travel_time_sec_frequency', fontsize=8)
# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# check the inner clustering performance by current K
time='hour_start'
df_plt=filtered_cluster.groupby(['inner_cluster', time])['inferred_travel_time_sec'].median().unstack().T
df_plt['total']=filtered_cluster.groupby(time)['inferred_travel_time_sec'].median()
df_plt.iloc[:,:-1].plot(marker='o')
df_plt['total'].plot(marker='*', color='black', label='total')
plt.ylabel('inferred_travel_time_sec (median)')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))

### 5.2 EDA for feature engineering

In [ ]:
df=filtered_cluster_updatedbyhour.copy()
df.head()

In [ ]:
#check the plot of inferred_traveltime_sec_updated vs. timestamp for the whole data:
df['inferred_traveltime_sec_updated'].plot()

In [ ]:
# check to see one month data by randomly picking one month
month_sample=5
df[df.index.month == month_sample]['inferred_traveltime_sec_updated'].plot()

In [ ]:
# check to see one week data by randomly picking one week
# Filter data for a specific week in May (e.g., May 8–14)
df["2023-05-08":"2023-05-14"]['inferred_traveltime_sec_updated'].plot()

In [ ]:
# check to see one day data by randomly picking one day
df['2023-05-01 00:00:00': '2023-05-01 23:00:00']['inferred_traveltime_sec_updated'].plot(marker='o')

Analysis:
- it seems the inferred_travel_time_sec shows a stable trend over time but appears a seasonal pattern with period of 24hours.

In [ ]:
# check the aggregated inferred_traveltime_sec_updated vs. months
df.groupby('month')['inferred_traveltime_sec_updated'].median().plot(kind='line', marker='*')

In [ ]:
# check the aggregated inferred_traveltime_sec_updated vs. day of week
df.groupby('dayname')['inferred_traveltime_sec_updated'].median().reindex([
    'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']).plot(kind='line', marker='*')

In [ ]:
# check the aggregated inferred_traveltime_sec_updated vs. hours
df.groupby('hour_start')['inferred_traveltime_sec_updated'].median().plot(kind='line', marker='*')

Analysis:
- although we can see some seasonal pattern of the inferred travel time by months and day of week, they are  insignificant by comparing with the pattern by hours.
- from the plot, we can divide a day into morning, afternoon, and night timeframes, and in each timeframe, there are two peak_hours as below:
morning=[6,7,8,9,10,11]
morning_peak=[7,8]
afternoon=[12,13,14,15,16,17]
afternoon_peak=[16, 17]
night=[18,19,20,21,22,23,0,1,2,3,4,5]
night_peak=[18,19]

In [ ]:
# check the completeness of temporal information
month_day_hour=df.groupby(['month','day'])['hour_start'].count().reset_index()
for month in range(1, 13):
    num_days=len(month_day_hour[month_day_hour['month']==month])
    avg_num_hours=round(month_day_hour[month_day_hour['month']==month]['hour_start'].mean(),2)
    min_num_hours=month_day_hour[month_day_hour['month']==month]['hour_start'].min()
    print(f'month {month}, num_days: {num_days}, avg_num_hours: {avg_num_hours}, min_num_hours: {min_num_hours}')

Analysis:
- It seems the hourly temporal data is incomplete. 
- we need to fill up the missing values first for an easier operation in the following (like to create the lag features)

#### 5.2.1 fill up missing hours

In [ ]:
# generate a full range of hours for each day
full_hours = (
   df.groupby(['year', 'month', 'day'])['hour_start']
    .apply(lambda x: pd.Series(np.arange(24)))
    .reset_index(level=-1, drop=True)  # Drop the original hours index from apply
    .reset_index()
)
full_hours

In [ ]:
# merge the full_hours with the original dataframe to find missing rows
df_fh = pd.merge(full_hours, df, on=['year', 'month', 'day', 'hour_start'], how='left')

In [ ]:
# fill na for weather related features, use forward-fill or backward-fill directly
weather_features=['temp', 'dwpt', 'rhum', 'prcp', 'wdir', 'pres']
df_fh[weather_features]=df_fh[weather_features].ffill().bfill()

In [ ]:
filtered_cluster_updatedbyhour_full[weather_features].isnull().sum()

In [ ]:
#create timeframe and peak_hours columns to indicate whether the current hour is from morning or afternoon or night and whether it is peak hour
df_fh['timeframe']=df_fh['hour_start'].apply(
    lambda x: 'morning' if 6<=x<12 else 'afternoon' if 12<=x<18 else 'night')
df_fh['peak_hours']=df_fh['hour_start'].apply(lambda x: 1 if x in [7,8,16,17,18,19] else 0)

In [ ]:
# fill na for inferred_traveltime_sec_updated based on peak_hours and non_peak_hours in different timeframes
# split into peak and non-peak sub datasets for each timeframe
morning_peak = df_fh[(df_fh['timeframe']=='morning') & (df_fh['peak_hours']==1)].copy()
morning_non_peak = df_fh[(df_fh['timeframe']=='morning') & (df_fh['peak_hours']==0)].copy()

afternoon_peak = df_fh[(df_fh['timeframe']=='afternoon') & (df_fh['peak_hours']==1)].copy()
afternoon_non_peak = df_fh[(df_fh['timeframe']=='afternoon') & (df_fh['peak_hours']==0)].copy()

night_peak = df_fh[(df_fh['timeframe']=='night') & (df_fh['peak_hours']==1)].copy()
night_non_peak = df_fh[(df_fh['timeframe']=='night') & (df_fh['peak_hours']==0)].copy()

# fill na of inferred_traveltime_sec_updated for peak hours
morning_peak['inferred_traveltime_sec_updated'] = morning_peak['inferred_traveltime_sec_updated'].ffill().bfill()
afternoon_peak['inferred_traveltime_sec_updated'] = afternoon_peak['inferred_traveltime_sec_updated'].ffill().bfill()
night_peak['inferred_traveltime_sec_updated'] = night_peak['inferred_traveltime_sec_updated'].ffill().bfill()
# fill na of inferred_traveltime_sec_updated for non-peak hours
morning_non_peak['inferred_traveltime_sec_updated'] = morning_non_peak['inferred_traveltime_sec_updated'].ffill().bfill()
afternoon_non_peak['inferred_traveltime_sec_updated'] = afternoon_non_peak['inferred_traveltime_sec_updated'].ffill().bfill()
night_non_peak['inferred_traveltime_sec_updated'] = night_non_peak['inferred_traveltime_sec_updated'].ffill().bfill()

# Combine the data
df_fh_filled = pd.concat([morning_peak, morning_non_peak, afternoon_peak, afternoon_non_peak, night_peak, night_non_peak]).sort_index()
df_fh_filled

In [ ]:
# fill na in 'dayname' column based on year, month, and day
df_fh_filled['dayname'] = (
    df_fh_filled.groupby(['year', 'month', 'day'])['dayname']
    .transform(lambda x: x.ffill().bfill())
)

In [ ]:
df_fh_filled

In [ ]:
df_fh_filled.isnull().sum()

#### 5.2.2 Generate lag and rolling features

In [ ]:
# rename 'hour_start' to 'hour'
df_fh_filled = df_fh_filled.rename(columns={'hour_start': 'hour'})

# create a datetime index
df_fh_filled['datetime'] = pd.to_datetime(df_fh_filled[['year', 'month', 'day', 'hour']])

# set 'datetime' as the index
df_fh_filled = df_fh_filled.set_index('datetime')

# sort the dataframe by the new datetime index
df_fh_filled = df_fh_filled.sort_index()

In [ ]:
# create a time series data
timeseries = df_fh_filled['inferred_traveltime_sec_updated']

# plot ACF and PACF
plt.figure(figsize=(15, 9))

plt.subplot(121)
plot_acf(timeseries, lags=24*60, ax=plt.gca()) # create 60 days autocorrelation
plt.title('Autocorrelation')

plt.subplot(122)
plot_pacf(timeseries, lags=24*3, ax=plt.gca()) # create 3 days partial autocorrelation
plt.title('Partial Autocorrelation')

plt.tight_layout()
plt.show()

Analysis:
- from the ACF plot, it seems the rolling window of 1 month is good enough since we can clearly see the second month is close to the confidence interval band.
- from the PCF plot, it seems lag1, lag2,  lag23, lag24 are good options since all the others are close to the confidence interval band.

In [ ]:
# Create lag features for inferred_traveltime_sec_updated
lags = [1, 2, 23, 24]
for lag in lags:
    df_fh_filled[f'lag_{lag}'] = df_fh_filled['inferred_traveltime_sec_updated'].shift(lag)
df_fh_filled

In [ ]:
# define rolling windows in hours
windows = {'1w': 7*24, '1m': 30*24}  # Weekly, Monthly (in hours)

# create rolling median for each window, also the rolling window only includes the same hour as the current hour
for window_name, window_size in windows.items():
    # rolling median
    df_fh_filled[f'rolling_median_{window_name}'] = (
        df_fh_filled.groupby('hour')['inferred_traveltime_sec_updated']
        .transform(lambda x: x.rolling(window=window_size, min_periods=1).median())
    )
    
df_fh_filled

In [ ]:
# since the number of rows with na values is insignificant, we just drop them directly 
df_fh_filled.dropna(inplace=True)
df_fh_filled

#### 5.2.3 EDA on individual features and correlation between features

##### 5.2.3.1 Exploring single numerical features

In [ ]:
df_fh_filled.info()

In [ ]:
num_features=df_fh_filled.select_dtypes(['float64'])
num_features

In [ ]:
num_features.hist(figsize=(16, 20), bins=100)

In [ ]:
# calculate the skewness for each feature
num_features.skew() 

Analysis:
- it seems the prcp feature is highly skewed, followed by the inferred_traveltime_sec_updated and its lag features.
- all weather related features show clear seasonal pattern.
- rolling_median_1w and rolling_median_1m have very similar distribution, they might be duplicated.

##### 5.2.3.2 Exploring the correlation between features

In [ ]:
# calculate the correlation matrix
corr = num_features.corr()

# plot the heatmap with annotations
plt.figure(figsize=(12, 6))
sns.heatmap(
    corr, 
    annot=True, 
    fmt=".2f", 
    cmap="coolwarm", 
    cbar=True, 
    annot_kws={"size": 10} 
)
plt.title("Correlation Matrix with Coefficients")
plt.show()

In [ ]:
#For a better visualization, we need to do some filtering to only display the high correlation values: 
plt.figure(figsize=(12, 6))
sns.heatmap(
    corr[abs(corr)>=0.3], 
    annot=True, 
    fmt=".2f", 
    cmap="coolwarm", 
    cbar=True, 
    annot_kws={"size": 10} 
)
plt.title("Correlation Matrix with good coefficients (>=0.3 or <=-0.3)")
plt.show()

In [ ]:
corr

In [ ]:
#if we want to see which features are highly correlated with the label, we can do: 
plt.figure(figsize=(12, 6))
sns.heatmap(
    corr[['inferred_traveltime_sec_updated']].iloc[1:], 
    annot=True, 
    fmt=".2f", 
    cmap="coolwarm", 
    cbar=True, 
    annot_kws={"size": 10} 
)
plt.title("Correlation between label (inferred travel time) and each other feature")
plt.show()

Analysis: from all above heatmaps, it seems
- the inferred_traveltime_sec_updated has insignificant relationship with all weather related features.
- the inferred_traveltime_sec_updated has stronger linear relationship with rolling_median_1w and rolling_median_1m, following by lag_1 and lag_24, then followed by lag_2 and lag_23.
- there exists a very strong linear relationship between rolling_median_1w and rolling_median_1m (over 99%), we can consider dropping one of them to avoid duplication.
- temp and dwpt also have a strong linear relationship (over 88%), we can consider dropping one of them.

In [ ]:
#check scatter plots between different features:
sns.pairplot(num_features, height=3, aspect=1.5) 
plt.gcf().set_size_inches(15, 15) 
plt.tight_layout() 
plt.show()

Analysis:
- clearly temp and dwpt, rolling_median_1w and rolling_medain_1m  are linearly correlated. 

In summary:
- we can drop rolling_median_1w while keepping rolling_median_1m (since rolling_median_1m has more data points to be aggregated).
- the features rolling_median_1m, lag_1, lag_24,  lag_2 and lag_23 showed relatively stronger linearly relationship with the target inferred travel time.
- temp and dwpt are highly correlated one of which can be dropped.
- all weather related feaures showed insignificant correlation with the target inferred travel time.

##### 5.2.3.3 Use Xgboost as the base model to do feature engneering

In [ ]:
data=df_fh_filled.copy()
#from the above analysis, we can drop rolling_median_1w and temp due to the duplication.
data.drop(['rolling_median_1w', 'temp'], axis=1, inplace=True)
data

In [ ]:
#convert the categorical features into 'category' dtype
data['dayname'] = data['dayname'].astype('category')
data['timeframe'] = data['timeframe'].astype('category')

# drop the insignificant features according to the feature importances analysis
## we want to drop 'lag_1' and 'lag_2' even they showed significant importances since they are difficult to implement for real-time prediction
data.drop(['year', 'month', 'rhum',  'prcp', 'lag_1', 'lag_2'], axis=1, inplace=True)

# split data into features and label  
X = data.drop('inferred_traveltime_sec_updated', axis=1)
y = data['inferred_traveltime_sec_updated']

# Train-test split (85% train, 15% test)
X_trainfull, X_test, y_trainfull, y_test = train_test_split(X, y, test_size=0.15, random_state=42, shuffle=False)  # shuffle=False for time-series

In [ ]:
# train-validation split (85% train, 15% validation in trainfull)
X_train, X_val, y_train, y_val = train_test_split(X_trainfull, y_trainfull, test_size=0.15, random_state=42, shuffle=False)

In [ ]:
# Initialize the model
xgb_model = xgb.XGBRegressor(
    n_estimators=500,
    learning_rate=0.003,
    max_depth=4,
    enable_categorical=True,
    random_state=42
)

# Train the model
xgb_model.fit(X_train, y_train)

# Predict on the validation set
y_val_pred = xgb_model.predict(X_val)

# Evaluate the model
mae=mean_absolute_error(y_val, y_val_pred)
me_ae=median_absolute_error(y_val, y_val_pred)
r2 = r2_score(y_val, y_val_pred)

print(f"Mean Absolute Error (MAE): {mae:.2f}sec")
print(f"Median Abosolute Error: {me_ae:.2f}sec")
print(f'r2_score: {r2:.2f}')

In [ ]:
# Plot feature importances
plt.figure(figsize=(10, 8))
xgb.plot_importance(xgb_model, importance_type='weight')
plt.title("Feature Importances")
plt.show()

Regon 1:
cluster 0: mean=6sec
with original features (only dropped rolling_median_1w and temp):
- Mean Absolute Error (MAE): 1.40sec
- Median Abosolute Error: 1.15sec
- r2_score: 0.18
after dropping ['year', 'month', 'rhum',  'pres', 'prcp', 'peak_hours'] features:
- Mean Absolute Error (MAE): 1.40sec
- Median Abosolute Error: 1.15sec
- r2_score: 0.18
after further droping ['lag_1', 'lag_2'] which are difficult to implement in reality:
- Mean Absolute Error (MAE): 1.42sec
- Median Abosolute Error: 1.14sec
- r2_score: 0.16
cluster 1: mean=38sec
with original features (only dropped rolling_median_1w and temp):
- Mean Absolute Error (MAE): 4.71sec
- Median Abosolute Error: 3.72sec
- r2_score: 0.44
after ['year',  'rhum',  'pres', 'prcp', 'peak_hours', 'lag_1', 'lag_2' ] features removal:
- Mean Absolute Error (MAE): 4.96sec
- Median Abosolute Error: 4.07sec
- r2_score: 0.41
cluster 2: mean=16sec
with original features (only dropped rolling_median_1w and temp):
- Mean Absolute Error (MAE): 1.84sec
- Median Abosolute Error: 1.49sec
- r2_score: 0.49
after ['year',  'rhum',  'pres', 'prcp', 'peak_hours', 'lag_1', 'lag_2' ] features removal:
- Mean Absolute Error (MAE): 1.85sec
- Median Abosolute Error: 1.53sec
- r2_score: 0.48
**cluster 3 (with around 40% missing hours): mean=66sec**
with original features (only dropped rolling_median_1w and temp):
- Mean Absolute Error (MAE): 10.77sec
- Median Abosolute Error: 8.24sec
- r2_score: 0.37
after ['year',  'rhum',  'pres', 'prcp',  'lag_1', 'lag_2' ] features removal:
- Mean Absolute Error (MAE): 11.90sec
- Median Abosolute Error: 9.87sec
- r2_score: 0.27
cluster 4: mean=27sec
with original features (only dropped rolling_median_1w):
- Mean Absolute Error (MAE): 3.50sec
- Median Abosolute Error: 2.98sec
- r2_score: 0.42
after ['year', 'month',  'rhum',  'prcp',  'lag_1', 'lag_2' ] features removal:
- Mean Absolute Error (MAE): 3.60sec
- Median Abosolute Error: 3.06sec
- r2_score: 0.40

Conclusion: we can just keep the following features for ML modeling:
- rolling_median_1m
- dayname
- hour
- lag_24
- lag_23
- dwpt
- timeframe
- day
- wdir

### 5.3 Machine Learning Modeling

#### 5.3.1 Xgboost model with Bayesian optimization (BayesSearchCV)

In [ ]:
data.columns

In [ ]:
# define time-series cross-validation
n_splits = 5  # number of splits for time-series CV
tscv = TimeSeriesSplit(n_splits=n_splits)

# define XGBoost model
xgb_model = xgb.XGBRegressor(enable_categorical=True, random_state=42)

# define hyperparameter search space
param_space = {
    'n_estimators': (100, 1000),  # Number of trees
    'learning_rate': (0.001, 0.1, 'log-uniform'),  # Learning rate
    'max_depth': (3, 10),  # Tree depth
    'subsample': (0.6, 1.0, 'uniform'),  # Fraction of samples
    'colsample_bytree': (0.6, 1.0, 'uniform')  # Fraction of features
}

# define custom scoring function (e.g., Mean Absolute Error)
scorer = make_scorer(mean_absolute_error, greater_is_better=False)

# set up Bayesian optimization with cross-validation
opt = BayesSearchCV(
    estimator=xgb_model,
    search_spaces=param_space,
    cv=tscv,
    n_iter=50,  # Number of iterations for optimization
    scoring=scorer,
    n_jobs=-1,
    verbose=3,
    random_state=42
)

# Fit the model with Bayesian optimization
opt.fit(X_trainfull, y_trainfull)
# Best hyperparameters and model
best_params = opt.best_params_
best_model = opt.best_estimator_

In [ ]:
# retrieve the cross-validation results specifically for the best estimator
# index of the best estimator
best_index = opt.best_index_

# results for the best estimator
best_mean_test_score = -opt.cv_results_['mean_test_score'][best_index]  # Negate to get positive MAE
best_std_test_score = opt.cv_results_['std_test_score'][best_index]

print("Best Parameters:", best_params)
print(f"Best Mean MAE: {best_mean_test_score:.2f} sec")
print(f"Best Std MAE: {best_std_test_score:.2f} sec")

Region 1:
- cluster 0:
Best Mean MAE: 1.52 sec
Best Std MAE: 0.08 sec
- cluster 1:
Best Mean MAE: 4.87 sec
Best Std MAE: 0.18 sec
- cluster 2:
Best Mean MAE: 1.98 sec
Best Std MAE: 0.10 sec
- **cluster 3 (more missing hours)**:
Best Mean MAE: 12.32 sec
Best Std MAE: 0.80 sec
- cluster 4:
Best Mean MAE: 3.63 sec
Best Std MAE: 0.12 sec

In [ ]:
# Save the best estimator to a file
joblib.dump(best_model, f'best_xgb_model_region{region}_cluster{cluster}.pkl')
# Load the model back when needed
#loaded_model = joblib.load('best_xgb_model_region1_cluster0.pkl')


In [ ]:
# Evaluate the best model on a test set
y_test_pred = best_model.predict(X_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_test_pred)
me_ae = median_absolute_error(y_test, y_test_pred)
r2 = r2_score(y_test, y_test_pred)

print(f"Mean Absolute Error (MAE): {mae:.2f} sec")
print(f"Median Absolute Error: {me_ae:.2f} sec")
print(f"R2 Score: {r2:.2f}")

#### 5.3.2 Prophet model

In [ ]:
# use df instead of data because prophet can automatically handel the missing hours, we can just use the original dataset instead of the one after 
## filling up the missing hours
df

In [ ]:
# drop insignificant features and make uniform hourly spacing
df_up=df.drop(['year', 'month', 'day', 'dayname', 'temp', 'rhum', 'prcp', 'pres'], axis=1)
df_hourly = df_up.reset_index().resample('h', on='trip_start_timestamp').mean().reset_index()
df_hourly

In [ ]:
# drop na after resampling
df_hourly.dropna(inplace=True)
df_cp=df_hourly.copy()
df_cp

In [ ]:
# prepare data for Prophet
df_prophet = df_cp.rename(columns={'trip_start_timestamp': 'ds', 'inferred_traveltime_sec_updated': 'y'})
df_prophet

In [ ]:
# select the external regressors 
regressors = ['dwpt', 'wdir', 'hour_start']
# scale the external regressors

# initialize the scaler
scaler = StandardScaler()

# fit and transform the regressors
scaled_regressors = scaler.fit_transform(df_prophet[regressors])

# add scaled regressors back to the dataframe
df_prophet[['dwpt_scaled', 'wdir_scaled', 'hour_scaled']] = scaled_regressors
df_prophet.head()

In [ ]:
#drop useless columns
df_prophet.drop(['hour_start', 'dwpt',  'wdir'], axis=1, inplace=True)
df_prophet

In [ ]:
# ensure the dataframe is ordered by ascending on datetime  
df_prophet = df_prophet.sort_values(by=['ds'])
df_prophet

In [ ]:
# split data into train and test subsets

# define the split point
split_point = '2024-02-17 00:00:00'  # 80% for training, 20% for testing

# Training and test datasets
test = df_prophet[df_prophet['ds']>=split_point]
train = df_prophet[df_prophet['ds']<split_point]

In [ ]:
train

In [ ]:
test

In [ ]:
len(test)/len(df_prophet)

In [ ]:
# perform cross-validation

# initialize Prophet model
model = Prophet()

# add scaled external regressors
model.add_regressor('dwpt_scaled')
model.add_regressor('wdir_scaled')
model.add_regressor('hour_scaled')

# fit the model to the training dataset
model.fit(train)  # for prophet objects initialization

# perform cross-validation
prophet_cv = cross_validation(
    model=model,
    initial='150 days',  # initial training period
    period='30 days',    # spacing between splits
    horizon='7 days'    # forecast horizon for test sets
)

# view the cross-validation results
prophet_cv

In [ ]:
# set the display option to show all rows
#pd.set_option('display.max_rows', None)
# get performance metrics
prophet_performance = performance_metrics(prophet_cv)
prophet_performance

In [ ]:
prophet_performance.set_index('horizon').describe().drop('count')

## **Model Comparison: Prophet vs. XGBoost for Region 1**

| **Cluster**   | **Prophet MAE** | **Prophet STD** | **XGBoost Best MAE** | **XGBoost Std of Best MAE** |
|---------------|-----------------|-----------------|---------------------------|--------------------------|
| **Cluster 0** | 1.52 sec        | 0.14 sec        | 1.52 sec                  | 0.08 sec                 |
| **Cluster 1** | 5.09 sec        | 0.71 sec        | 4.87 sec                  | 0.18 sec                 |
| **Cluster 2** | 2.13 sec        | 0.21 sec        | 1.98 sec                  | 0.10 sec                 |
| **Cluster 3** | **13.30 sec**   | **2.00 sec**    | **12.32 sec**             | **0.80 sec**             |
| **Cluster 4** | 3.71 sec        | 0.39 sec        | 3.63 sec                  | 0.12 sec                 |


In [ ]:
# plot MAE over horizons
plt.figure(figsize=(10, 6))
prophet_performance['mae'].plot(marker='o')
plt.title('Mean Absolute Error in sec Across Forecast Horizons')
plt.xlabel('Horizon (hours)')
plt.ylabel('MAE (sec)')

In [ ]:
# plot the actual values and prediction intervals
plt.figure(figsize=(10, 6))
plt.scatter(prophet_cv.index, prophet_cv['y'], label='Actual Values', marker='o')
plt.fill_between(prophet_cv.index, prophet_cv['yhat_lower'], prophet_cv['yhat_upper'], color='gray', alpha=0.3, label='Confidence Interval')
plt.legend()
plt.title('Prediction Intervals and Actual Values')
plt.xlabel('Observation Index')
plt.ylabel('inferred_traveltime_sec_updated')

In [ ]:
test

In [ ]:
# refit the model on the entire training dataset:
final_model = Prophet()
final_model.fit(train)

# predict on the test dataset:
future = test
forecast = final_model.predict(future)

In [ ]:
test_merge=pd.merge(test, forecast[['ds', 'yhat']], on=['ds']) 
# evaluate the model
mae=mean_absolute_error(test_merge['y'], test_merge['yhat'])
r2 = r2_score(test_merge['y'], test_merge['yhat'])

print(f"Mean Absolute Error (MAE): {mae:.2f}sec")
print(f'r2_score: {r2:.2f}')


In [ ]:
test_merge

In [ ]:
test_regions_clusters={}

In [ ]:
test_regions_clusters[f'region_{region}_cluster_{cluster}']=test_merge

In [ ]:
# Save the dictionary to a file
joblib.dump(test_regions_clusters, f'test_region{region}_clusters.pkl')

In [ ]:
# Load the dictionary from the file
test_regions_clusters = joblib.load(f'test_region{region}_clusters.pkl')

### 5.4 Use the prediction results to do route optimization and compare the results with that based on the inferred travel time

#### 5.4.1 Merge the prediction results for each cluster to the original regional dataset

In [ ]:
# select a region dataset to display
region=1
region_withcluster=pd.read_csv(f'region_{region}_withcluster.csv')
region_withcluster.head()

In [ ]:
region_withcluster.head()

In [ ]:
region_withcluster.columns

In [ ]:
# add 'day' column
region_withcluster['trip_start_timestamp']=pd.to_datetime(region_withcluster['trip_start_timestamp'])
region_withcluster['day']=region_withcluster['trip_start_timestamp'].dt.day

In [ ]:
region_withcluster.columns

In [ ]:
# add 'datetime' column that has a uniform hourly spacing so as to do merge with the prediction results that with uniform hourly spacing
region_withcluster.rename(columns={'hour_start': 'hour'}, inplace=True)
region_withcluster['datetime']=pd.to_datetime(region_withcluster[['year', 'month', 'day', 'hour']])

In [ ]:
# groupby datetime for each road segment to create a dataframe with uique hourly datetime for each segment
region_data_groupbytime=region_withcluster.groupby(['u', 'v', 'datetime'])[['speed_kph', 'travel_time',  'inferred_travel_time_sec', 
                                                                            'year', 'month', 'day', 'hour', 'cluster']].mean().reset_index()

In [ ]:
region_data_groupbytime

In [ ]:
# merge with our prediction results
split_point = '2024-02-17 00:00:00'
region_data_groupbytime_test=region_data_groupbytime[region_data_groupbytime['datetime']>=split_point].sort_values(by=['datetime'])

#remember to update num_clusters for different regions
num_clusters=5

region_data_ls=[]
for cluster in range(num_clusters):
    test_regions_clusters[f'region_{region}_cluster_{cluster}']['cluster']=cluster
    region_data=pd.merge(region_data_groupbytime_test, test_regions_clusters[f'region_{region}_cluster_{cluster}'][['ds', 'cluster', 'yhat']], 
                         left_on=['datetime', 'cluster'], right_on=['ds', 'cluster'])
    region_data_ls.append(region_data)
region_data_merge=pd.concat(region_data_ls)    

In [ ]:
region_data_merge

#### 5.4.2 Compare the route optimization results by using the predition results on cluster level and the inferred travel time on segment level

##### 5.4.2.1 Analyze the difference between the inferred_travel_time_sec and the predicted travel time for each segment

In [ ]:
# analyze the difference between the inferred_travel_time_sec and the prediction for each segment 
region_data_merge['abs_diff_sec_actual_pred']=abs(region_data_merge['yhat']-region_data_merge['inferred_travel_time_sec'])

In [ ]:
region_data_merge['abs_diff_sec_actual_pred'].plot(kind='hist', bins=100)

In [ ]:
region_data_merge['abs_diff_sec_actual_pred'].describe().drop('count')

##### 5.4.2.2 Compare the route optimization results by the prediction and the inferred travel time

In [ ]:
#create an OSMnx graph for Chicago city
place = 'Chicago, Illinois, USA'
G = ox.graph_from_place(place, network_type='drive')
# create geo-dataframe for both nodes and edges 
nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)

# add geometry from edges to G for plot in the following
for idx, row in edges.reset_index().iterrows():
    u, v, key = row['u'], row['v'], row['key']
    G[u][v][key]['geometry'] = row['geometry']

In [ ]:
#define a function to create a directed graph based on the given dataset.
def create_subset_graph_with_given_data(G, data):
    # Extract relevant edges (u, v) from the given data
    relevant_edges = [(row['u'], row['v']) for _, row in data.iterrows()]
    relevant_nodes = set([node for edge in relevant_edges for node in edge])

    # Handle multi-edge graph by selecting the shortest edge for each (u, v)
    subset_edges = []
    for u, v in relevant_edges:
        if G.has_edge(u, v):  
            # Find the edge with the minimum 'length' 
            min_edge_key = min(G[u][v], key=lambda k: G[u][v][k].get('length', float('inf')))
            subset_edges.append((u, v, min_edge_key))

    # Create the subset graph with the selected edges
    G_c = G.edge_subgraph(subset_edges).copy()

    # Add the inferred_travel_time_sec and yhat attributes to the edges in G_c
    for _, row in data.iterrows():
        u, v = row['u'], row['v']
        edge_keys = list(G_c[u][v].keys())  
        first_key = edge_keys[0]  # Get the first key
        G_c[u][v][first_key]['inferred_travel_time_sec'] = row['inferred_travel_time_sec']
        G_c[u][v][first_key]['prediction_in_sec'] = row['yhat']
     

    # Ensure all relevant nodes are included
    for node in relevant_nodes:
        if node not in G_c.nodes and node in G:
            G_c.add_node(node, **G.nodes[node])

    return G_c

In [ ]:
region_data_merge['datetime'].value_counts()

In [ ]:
# pick a specific time between '2024-02-17 00:00:00' and  '2024-04-30 23:00:00'
timestamp='2024-02-23 11:00:00'
region_data_merge_specifictime=region_data_merge[region_data_merge['datetime']==timestamp]
# create a subgraph corresponding to the specified timestamp
G_c = create_subset_graph_with_given_data(G, region_data_merge_specifictime)
print(f"Subset graph has {G_c.number_of_nodes()} nodes and {G_c.number_of_edges()} edges.")

In [ ]:
# Example to inspect an edge's attributes
u, v, key = next(iter(G_c.edges))
print(f"Edge ({u}, {v}) attributes: {G_c[u][v][key]}")

In [ ]:
# define a function to find pairs of nodes (start_node, end_node) with multiple routes
def find_pairs_with_multiple_routes(data, G_c, num_pairs=1000):   
    # initialize an empty list to store the pairs
    node_pairs = []
    # get unique list of nodes
    unique_nodes = list(set(G_c.nodes))  
    # iterate through unique pairs of nodes
    for i, start_node in enumerate(unique_nodes):
        for end_node in unique_nodes[i + 1:]:  # check unique pairs (start_node, end_node)
            # Find all simple paths between the nodes
            # I chose 80 as cutoff was based on the fact that the average trip_duration from the existing dataset is 12min, 80 cutoff  similuates this reality
            paths = list(nx.all_simple_paths(G_c, source=start_node, target=end_node, cutoff=80))  # Limit to paths of max 80 edges
            if len(paths) > 1:  # Check if there are multiple paths
                node_pairs.append((start_node, end_node))    
            # Stop when we have enough pairs
            if len(node_pairs) >= num_pairs:
                return node_pairs    
    # If less than `num_pairs` are found, return what we have
    return node_pairs

In [ ]:
node_pairs = find_pairs_with_multiple_routes(region_data_merge_specifictime, G_c)
len(node_pairs)

In [ ]:
# define a function to calculate the shortest route in a graph based on a given weight
def calculate_shortest_route(G_c, start_node, end_node, weight):

    # calculate the shortest path
    path = nx.shortest_path(G_c, source=start_node, target=end_node, weight=weight)
    
    # calculate the total weight of the path
    total_time = nx.shortest_path_length(G_c, source=start_node, target=end_node, weight=weight)
    
    return path, total_time

In [ ]:
path_inferred_ls=[]
path_prediction_ls=[]
time_inferred_ls=[]
#abs_timediff_percent_predtime_inferredtime means the absolute percentage error between the predicted travel time and the actual inferred total travel time. 
abs_timediff_percent_predtime_inferredtime_ls=[]
#abs_timediff_percent__predpath_inferredpath means the absolute percentage difference in travel time between the predicted path and the actual fastest path 
abs_timediff_percent_predpath_inferredpath_ls=[]
#abs_timediff_sec_predpath_inferredpath means the absolute travel time difference in sec between the predicted shortest path and the actual fastest path. 
abs_timediff_sec_predpath_inferredpath_ls=[]
#abs_timediff_sec_predtime_inferredtime means the absolute difference in sec between the predicted travel time and the actual inferred total travel time. 
abs_timediff_sec_predtime_inferredtime_ls=[]
comp_ls=[]
for i in range(len(node_pairs)):
   # shortest route based on inferred traffic time
    path_inferred, time_inferred = calculate_shortest_route(G_c, node_pairs[i][0], node_pairs[i][1], weight='inferred_travel_time_sec')
    path_inferred_ls.append(path_inferred)
    time_inferred_ls.append(time_inferred)
   # shortest route based on predicted travel time
    path_prediction, time_prediction = calculate_shortest_route(G_c, node_pairs[i][0], node_pairs[i][1], weight='prediction_in_sec')
    path_prediction_ls.append(path_prediction)
    comp_ls.append(path_inferred==path_prediction)
   # calculate the total inferred travel time for the calculated shortest route based on predicted travel time
    inferred_travel_time_fromprediction = 0
    for i in range(len(path_prediction) - 1): 
        u = path_prediction[i]  
        v = path_prediction[i + 1]
        edge_keys = list(G_c[u][v].keys())  
        first_key = edge_keys[0]  
        inferred_travel_time_fromprediction += G_c[u][v][first_key]['inferred_travel_time_sec']
    abs_timediff_percent_predpath_inferredpath_ls.append(abs(inferred_travel_time_fromprediction-time_inferred)/time_inferred)
    abs_timediff_percent_predtime_inferredtime_ls.append(abs(time_prediction-inferred_travel_time_fromprediction)/inferred_travel_time_fromprediction)
    abs_timediff_sec_predpath_inferredpath_ls.append(abs(inferred_travel_time_fromprediction-time_inferred))
    abs_timediff_sec_predtime_inferredtime_ls.append(abs(time_prediction-inferred_travel_time_fromprediction))

In [ ]:
# check the average trip_duration from the generated pairs
np.mean(time_inferred_ls)/60

In [ ]:
prediction_perf=pd.DataFrame({'absolute percentage time difference between the predicted path and the actual fastest path': 
                              abs_timediff_percent_predpath_inferredpath_ls, 'absolute percentage error between the predicted travel time and the actual inferred travel time':
              abs_timediff_percent_predtime_inferredtime_ls,'absolute time difference in sec between the predicted path and the actual fastest path': 
                              abs_timediff_sec_predpath_inferredpath_ls, 'absolute time difference in sec between the predicted travel time and the actual inferred travel time':
              abs_timediff_sec_predtime_inferredtime_ls})
prediction_perf


In [ ]:
prediction_perf.describe()

In [ ]:
# percentage of the shortest route calculated by prediction matches with that by inferred travel time
len(list(filter(lambda x: True if x is True else False, comp_ls)))/len(comp_ls)

In [ ]:
# Define a function to display the calculated shortest routes on a map with traffic info
def display_shortest_routes_with_traveltime_info(G_c, start_node, end_node):

    # Initialize map centered at the start node
    start_coords = (G_c.nodes[start_node]['y'], G_c.nodes[start_node]['x'])  # (latitude, longitude)
    m = folium.Map(location=start_coords, zoom_start=13)

    # Define weights and their visualization styles
    route_styles = {
        'inferred_travel_time_sec': {'color': '#E65100', 'dash_array': None},  # Solid line
        'prediction_in_sec': {'color': '#E65100', 'dash_array': '5,10'},  # Dashed line
    }

    total_times = {}  # Dictionary to store total travel times for each weight

    for weight, style in route_styles.items():
        # calculate shortest path and its nodes
        path = nx.shortest_path(G_c, source=start_node, target=end_node, weight=weight)
        
        # initialize total travel time
        total_time = 0

        # extract edges and their corresponding traffic information
        for u, v in zip(path[:-1], path[1:]):  # iterate over consecutive nodes in the path
            edge_data = G_c[u][v]  # access edge data
            edge_keys = list(G_c[u][v].keys())
            first_key = edge_keys[0]  # get the first key
            edge_geometry = edge_data[first_key].get("geometry")

            # extract traffic information
            inferred_time = edge_data[first_key].get("inferred_travel_time_sec", 0)
            prediction_time = edge_data[first_key].get("prediction_in_sec", 0)

            # add time to the total for the current weight
            travel_time = edge_data[first_key].get(weight, 0)  
            total_time += travel_time

            # add the edge as a polyline to the map
 
            line_coords = list(edge_geometry.coords)
            line_coords_reversed = [[lat, lon] for lon, lat in line_coords]  # Reverse for folium
            folium.PolyLine(
                locations=line_coords_reversed,
                color=style['color'],
                weight=5,
                dash_array=style['dash_array'],
                tooltip=f"<b>Route by {weight}</b><br>"
                            f"Start: {u}, End: {v}<br>"
                            f"Inferred Time: {inferred_time} sec<br>"
                            f"Predicted Time: {prediction_time} sec<br>"
                ).add_to(m)

        # store the total travel time in minutes
        total_times[weight] = total_time / 60  # convert seconds to minutes

    # add markers for start and end nodes
    folium.Marker(location=start_coords, popup="Start Node", icon=folium.Icon(color="blue")).add_to(m)
    end_coords = (G_c.nodes[end_node]['y'], G_c.nodes[end_node]['x'])

    # add total travel time popup at the End Node
    total_time_popup = "<br>".join(
        [f"<b>Route by {weight}</b>: {total_time:.2f} min" for weight, total_time in total_times.items()]
    )
    folium.Marker(
        location=end_coords,
        popup=f"<b>End Node</b><br>{total_time_popup}",
        icon=folium.Icon(color="blue")
    ).add_to(m)

    # Add title
    title_html = f"""
        <h3 align="center" style="font-size:16px"><b>Region {region} Route Optimization Comparison at 2024-02-23 11:00:00 am<br></h3>
        """
    m.get_root().html.add_child(folium.Element(title_html))

    return m


# Example usage
map_routes = display_shortest_routes_with_traveltime_info(G_c, node_pairs[2][0], node_pairs[2][1])
map_routes.save("shortest_routes_comparison.html")  # Save as an interactive HTML
map_routes
